In [1]:
import sys 
import os
import requests
import pandas as pd
import numpy as np
from pathlib import Path 
from PIL import Image
from skimage import io
import shutil

print(pd.__version__)
print(np.__version__)
!python -V

dir_meta = Path('meta')
dir_meta.mkdir(exist_ok=True, parents=True)


1.4.1
1.22.2
Python 3.9.7


In [2]:
# import pandas as pd
# df_meta_all = pd.read_csv('meta/meta_all.csv',index_col=0)
# df_meta_all

In [3]:
from distributed import Client
client = Client()
import modin.pandas as pd

# 建立jpg_top100目錄內meta檔
- meta data檔來源為ebird資料

#### ebird metat data 資料儲存方式
- ebird meta data位於downloaded/csv內
- 檔案目錄格式為 '科/學名_流水號.csv'
- 每個csv檔內容為該'學名_流水號'的所有影像的完整meta data

#### 處理流程
- 以'學名_流水號'(搭配1個獨立csv檔)為單位進行資料合併 
    - 以key(科、學名_流水號)讀取目前資料、對應的csv檔
    - 合併目前資料(Size、Filename)、對應的csv檔(所有metadat欄位)
    - 得到df_學名_流水號流水號檔案



Peudo code :
```
dir_ebirdmeta = Path(files_ebird_metadata)

query_f_sp = ['Family':'SciName_N', ...]

df_meta
for f, values in query_f_sp.keys():

    for sp in values:
        path_ebirdmeta =  dir_ebirdmeta.joinpath(f, sp + '.csv') 
        ebird_meta = pd.read_csv(path_ebirdmeta)
        jpg_top100_sp = jpg_top100.query(sp)

        df_meta_tmp = merge(jpg_top100_sp, ebird_meta, on=[f,sp])
        df_meta = pd.concat([df_meta, df_meta_tmp])

```



#### 讀取terminal指令輸出的jpg_top100目錄檔
-  `find downloaded/jpg_top100 -name "*.jpg" -type f -exec ls -l {} \; > jpg_top100.txt`

In [3]:

df_jpg100 = pd.read_csv('meta/jpg_top100.txt',
                        usecols=[4,8],
                        names=['Size', 'Path'],
                        sep=' ',
                        header=None 
                        )
# df_jpg100 = df_jpg100.query('~Path.str.contains("checkpoint")').reset_index(drop=True)
df_jpg100

,Size,Path
0,162891,downloaded/jpg_top100/Acanthisittidae/Acanthis...
1,296210,downloaded/jpg_top100/Acanthisittidae/Acanthis...
2,124934,downloaded/jpg_top100/Acanthisittidae/Acanthis...
3,294646,downloaded/jpg_top100/Acanthisittidae/Xenicus_...
4,230262,downloaded/jpg_top100/Acanthisittidae/Xenicus_...
...,...,...
1019966,211764,downloaded/jpg_top100/Zosteropidae/Zosterops_m...
1019967,109798,downloaded/jpg_top100/Zosteropidae/Zosterops_l...
1019968,206092,downloaded/jpg_top100/Zosteropidae/Zosterops_l...
1019969,165973,downloaded/jpg_top100/Zosteropidae/Zosterops_s...


In [4]:
def get_family(s:pd.Series):
    return Path(s).parent.name
def get_basename(s:pd.Series):
    return Path(s).stem
def get_Dir_FName(s:pd.Series):
    return Path(s).parent.name + '/' + Path(s).stem

In [5]:

df_jpg100 = df_jpg100.query('~Path.str.contains("checkpoint")').reset_index(drop=True)
df_jpg100['Family'] = df_jpg100.Path.apply(get_family)
# df_jpg100['Filename'] = df_jpg100.Path.apply(get_basename)
df_jpg100['Dir_FName'] = df_jpg100.Path.apply(get_Dir_FName)

df_jpg100['ML Catalog Number'] = df_jpg100.Dir_FName.apply(lambda x: x.split('_')[-1]).astype('uint32')
df_jpg100['Sci_N'] = df_jpg100.Dir_FName.str.extract(r'([\w]+_[a-z]+_[\d]{1,3})')
# convert byte to kb
df_jpg100['Size'] = df_jpg100.Size.apply(lambda x: round(x/1024,2))                     

df_jpg100 = df_jpg100[['Family', 'Sci_N','ML Catalog Number','Dir_FName', 'Size']]

df_jpg100 = df_jpg100.sort_values(by=['Family','Sci_N','ML Catalog Number']).reset_index(drop=True)

df_jpg100 

To request implementation, send an email to feature_requests@modin.org.


,Family,Sci_N,ML Catalog Number,Dir_FName,Size
0,Acanthisittidae,Acanthisitta_chloris_0,22413261,Acanthisittidae/Acanthisitta_chloris_0_22413261,61.48
1,Acanthisittidae,Acanthisitta_chloris_0,22413271,Acanthisittidae/Acanthisitta_chloris_0_22413271,68.64
2,Acanthisittidae,Acanthisitta_chloris_0,30352591,Acanthisittidae/Acanthisitta_chloris_0_30352591,132.18
3,Acanthisittidae,Acanthisitta_chloris_0,40629251,Acanthisittidae/Acanthisitta_chloris_0_40629251,176.57
4,Acanthisittidae,Acanthisitta_chloris_0,41060091,Acanthisittidae/Acanthisitta_chloris_0_41060091,165.23
...,...,...,...,...,...
1019963,Zosteropidae,Zosterornis_whiteheadi_0,219727771,Zosteropidae/Zosterornis_whiteheadi_0_219727771,310.95
1019964,Zosteropidae,Zosterornis_whiteheadi_0,219727781,Zosteropidae/Zosterornis_whiteheadi_0_219727781,349.39
1019965,Zosteropidae,Zosterornis_whiteheadi_0,219727791,Zosteropidae/Zosterornis_whiteheadi_0_219727791,368.17
1019966,Zosteropidae,Zosterornis_whiteheadi_0,219727801,Zosteropidae/Zosterornis_whiteheadi_0_219727801,324.02


#### 建立metadata檔

In [6]:
df_f_sp = df_jpg100.groupby(['Family','Sci_N']).Dir_FName.count().reset_index()[['Family','Sci_N']]
df_f_sp

,Family,Sci_N
0,Acanthisittidae,Acanthisitta_chloris_0
1,Acanthisittidae,Xenicus_gilviventris_0
2,Acanthizidae,Acanthiza_apicalis_0
3,Acanthizidae,Acanthiza_chrysorrhoa_0
4,Acanthizidae,Acanthiza_cinerea_0
...,...,...
16398,Zosteropidae,Zosterornis_hypogrammicus_0
16399,Zosteropidae,Zosterornis_latistriatus_0
16400,Zosteropidae,Zosterornis_nigrorum_0
16401,Zosteropidae,Zosterornis_striatus_0


In [187]:
# dir_meta = Path('downloaded/csv')

# # query_f_sp = {'Family':'SciName_N', ...}
# sp_total = len(df_f_sp)

# for idx, (id, (f, s_n)) in enumerate(df_f_sp.iterrows()):
#     # if idx==2: break
    
#     path_meta =  dir_meta.joinpath(f, s_n + '.csv') 
    
#     ebird_meta = pd.read_csv(path_meta)

#     # ebird_meta = ebird_meta.loc[:, ~ebird_meta.columns.str.contains("Family")]  # select all columns excloud 

#     jpg_top100_sp = df_jpg100.query('Sci_N==@s_n')[['ML Catalog Number', 'Dir_FName','Size']]

#     df_meta_tmp = pd.merge(jpg_top100_sp, ebird_meta, on=['ML Catalog Number'], how='left')
    
#     if idx==0:
#         df_meta = df_meta_tmp 
#     else:
#         df_meta = pd.concat([df_meta, df_meta_tmp])
    
#     print(f'{idx:4d}/{sp_total} | Data merged : {len(df_meta):6,d} | {f:10s}, {s_n:20s}\t\t', end='\r')
    
# df_meta.reset_index(drop=True, inplace=True)
# df_meta.to_csv('meta/jpg_top100_meta.csv')
# print(f'\n"jpg_top100_meta.csv" saved')

In [12]:
df_meta = pd.read_csv('meta/jpg_top100_meta.csv', index_col=0)
 
print(len(df_meta))
print(len(df_jpg100))
df_meta.info()


Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


1019968
1019968
<class 'modin.pandas.dataframe.DataFrame'>
Int64Index: 1019968 entries, 0 to 1019967
Data columns (total 60 columns):
 #   Column                       Non-Null Count    Dtype  
---  ---------------------------  ----------------  -----  
 0   ML Catalog Number            1019968 non-null  int64
 1   Dir_FName                    1019968 non-null  object
 2   Filename                     1019968 non-null  object
 3   Sci_N                        1019968 non-null  object
 4   Size                         1019968 non-null  float64
 5   Scientific Name              1019924 non-null  object
 6   Common Name                  1019924 non-null  object
 7   Background Species           2427 non-null     object
 8   Recordist                    1019874 non-null  object
 9   Date                         1006617 non-null  object
 10  Year                         1006617 non-null  float64
 11  Month                        1006617 non-null  float64
 12  Day                          10

### 處理無法合併meta data的資料
- 檢查mlcn編碼無法對上的資料

In [216]:
# df_miss = (pd.concat([
#             df_jpg100[['Filename']], 
#             df_meta[['Filename']]
#             ])
#            .drop_duplicates(keep=False)
#            .merge(df_jpg100, on='Filename')
#            )
# df_miss

,Family,Sci_N,ML Catalog Number,Dir_FName,Size


- 有44筆資料在csv檔中查不到mlcn編碼而無法match

- Debug流程
    - 重新讀取csv及比對jpg_top100內的流水號編號
    - 資料筆數、 差集(jpg_top100 -csv )



In [280]:
# sp_mlcns_ebird = {}
# sp_mlcns_jpg100 = {}

# for id, rows in df_miss.iterrows():
#     name, family, sp, mlcn, _ = rows
#     print(id, name, family, sp, mlcn, end='\r')
    
#     path_meta =  dir_meta.joinpath(family, sp + '.csv') 
#     ebird_meta = pd.read_csv(path_meta)
#     mlcns_ebird = ebird_meta['ML Catalog Number'].sort_values().values
    
#     mlcns_jpg100 = df_jpg100.query('SciName==@sp')['ML Catalog Number'].sort_values().values
    
#     sp_mlcns_ebird[sp] = mlcns_ebird
#     sp_mlcns_jpg100[sp] = mlcns_jpg100


In [332]:
df_miss_mlcn = pd.merge(pd.DataFrame(sp_mlcns_ebird.items(), columns=['SciName','mlcns_ebird']),
                        pd.DataFrame(sp_mlcns_jpg100.items(), columns=['SciName','mlcns_jpg100'])
                        ).sort_values(by='SciName')
df_miss_mlcn['mlcns_ebird_c'] = df_miss_mlcn.mlcns_ebird.apply(lambda x : len(x) )
df_miss_mlcn['mlcns_jpg100_c'] = df_miss_mlcn.mlcns_jpg100.apply(lambda x : len(x) )
df_miss_mlcn['Diff'] = df_miss_mlcn.eval('mlcns_jpg100_c-mlcns_ebird_c')
df_miss_mlcn['Diff_mlcns'] = df_miss_mlcn.mlcns_jpg100.apply(lambda x: set(x)) - df_miss_mlcn.mlcns_ebird.apply(lambda x: set(x))
df_miss_mlcn

,SciName,mlcns_ebird,mlcns_jpg100,mlcns_ebird_c,mlcns_jpg100_c,Diff,Diff_mlcns
0,Alaudala_cheleensis_0,"[711413, 20815941, 20815951, 20815961, 2081597...","[711413, 20815941, 20815951, 20815961, 2081597...",95,96,1,{236857561}
8,Dysithamnus_xanthopterus_0,"[30062351, 36363481, 37121291, 41001351, 41001...","[30062351, 36363481, 37121291, 41001351, 41001...",90,91,1,{250630161}
12,Elaenia_pelzelni_0,"[703347, 37799121, 37876831, 46353171, 4635323...","[703347, 37799121, 37876831, 46353171, 4635323...",38,39,1,{160496841}
13,Empidonax_oberholseri_1,"[29995201, 33472671, 33472731, 36276381, 36276...","[29995201, 33472671, 33472731, 36276381, 36276...",98,100,2,"{250281361, 250281371}"
14,Empidonax_traillii_3,"[30507741, 30541201, 30541221, 30541261, 30541...","[30507741, 30541201, 30541221, 30541261, 30541...",48,49,1,{113563121}
15,Empidonax_wrightii_1,"[29995201, 33472671, 33472731, 36276381, 36276...","[29995201, 33472671, 33472731, 36276381, 36276...",98,100,2,"{250281361, 250281371}"
4,Francolinus_francolinus_2,"[206061091, 206063551, 206066351, 206066631, 2...","[206061091, 206063551, 206066351, 206066631, 2...",6,7,1,{255624301}
16,Hemitriccus_spodiops_0,"[35531191, 68629791, 68629801, 73092881, 12603...","[35531191, 68629791, 68629801, 73092881, 12603...",12,17,5,"{222412391, 204082381, 222412431, 222412371, 2..."
17,Mionectes_macconnelli_0,"[25677811, 27371701, 39590971, 39590981, 39595...","[25677811, 27371701, 39590971, 39590981, 39595...",50,55,5,"{222411491, 222411541, 222411481, 222411451, 2..."
18,Muscisaxicola_rufivertex_3,"[204125371, 262472881, 262472891]","[204125371, 205696561, 262472881, 262472891]",3,4,1,{205696561}


In [388]:
df_miss_mlcn.set_index('SciName').iloc[:,2:]

,mlcns_ebird_c,mlcns_jpg100_c,Diff,Diff_mlcns
SciName,,,,
Alaudala_cheleensis_0,95,96,1,{236857561}
Dysithamnus_xanthopterus_0,90,91,1,{250630161}
Elaenia_pelzelni_0,38,39,1,{160496841}
Empidonax_oberholseri_1,98,100,2,"{250281361, 250281371}"
Empidonax_traillii_3,48,49,1,{113563121}
Empidonax_wrightii_1,98,100,2,"{250281361, 250281371}"
Francolinus_francolinus_2,6,7,1,{255624301}
Hemitriccus_spodiops_0,12,17,5,"{222412391, 204082381, 222412431, 222412371, 2..."
Mionectes_macconnelli_0,50,55,5,"{222411491, 222411541, 222411481, 222411451, 2..."


#### mlcn缺失發生原因推測
- 部分mlcn可在同一學名但不同流水號編碼下找到
- 部分流水號則完全無法在csv檔中找出
- 部分資料因為在ebird網站改為不公開

#### 解決方案
- 根據mlcn上ebird網站查詢所屬對應的學名與科別
    - 根據對應的科名及學名開啟對應的csv檔核對meta資料
- ~~補上merge失敗nan的資料，維持資料筆數一致~~
- ~~從原本採用 '學名_編號'改採 讀取該學名所有bcsv檔~~


In [3]:
# df_miss = df_meta[df_meta['Scientific Name'].isnull()].loc[:,:'Size']
# df_miss.to_csv('meta/mlcn_miss.csv')
df_miss = pd.read_csv('meta/mlcn_miss.csv', index_col=0)
print(len(df_miss))
df_miss.head()

44


,ML Catalog Number,Dir_FName,Filename,Sci_N,Size
1019924,236857561,Alaudidae/Alaudala_cheleensis_0_236857561,Alaudala_cheleensis_0_236857561,Alaudala_cheleensis_0,46.20
1019925,135237181,Furnariidae/Phacellodomus_rufifrons_1_135237181,Phacellodomus_rufifrons_1_135237181,Phacellodomus_rufifrons_1,116.29
1019926,125398821,Furnariidae/Synallaxis_courseni_0_125398821,Synallaxis_courseni_0_125398821,Synallaxis_courseni_0,443.20
1019927,204080901,Furnariidae/Syndactyla_striata_0_204080901,Syndactyla_striata_0_204080901,Syndactyla_striata_0,136.33
1019928,204080911,Furnariidae/Syndactyla_striata_0_204080911,Syndactyla_striata_0_204080911,Syndactyla_striata_0,147.83


In [4]:
mlcn_missed =  df_miss['ML Catalog Number'].sort_values().unique()
print(len(mlcn_missed))
mlcn_missed

To request implementation, send an email to feature_requests@modin.org.


38


array([ 35118661,  43334941,  43334951,  43334961,  43334971,  82069831,
        82069851, 113563121, 125398821, 135237181, 160496841, 191069511,
       191069541, 204080901, 204080911, 204082381, 204082391, 205696561,
       222411451, 222411481, 222411491, 222411541, 222411551, 222412371,
       222412391, 222412431, 236857561, 250281361, 250281371, 250630161,
       253995111, 255624301, 257279341, 257279351, 257279481, 257279491,
       257279531, 257279541], dtype=int64)

- 讀取網頁資料
    - 依據mlcn值直接到網站查詢，目前使用的學名  https://macaulaylibrary.org/asset/'mlcn'
    - 資料填寫至'meta/mlcn_missed_checklist.csv'

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# sp_mlcn = {}
# for idx, mlcn in enumerate(mlcn_missed):
#     url = f'https://macaulaylibrary.org/asset/{mlcn}'
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text)
#     try:
#         sci_name = soup.find('h5').text
#     except Exception as err:
#         # print(err)
#         sci_name = soup.find('p').text
        
#     sp_mlcn[mlcn] = sci_name
#     print(f'{idx}, {mlcn:10d}, {sci_name}', end='\r')

#     time.sleep(0.5)
# sp_mlcn 

In [98]:
# df_miss_SciName = pd.DataFrame(sp_mlcn.items(), columns=['ML Catalog Number', 'Scientific Name'])

# state = 'This ML catalog number is not publicly available.'
# df_miss_SciName['Scientific Name'] = df_miss_SciName['Scientific Name'].replace(state, np.nan)

# con = df_miss_SciName['Scientific Name'].notna()
# df_miss_SciName['Macaulay_public'] = np.where(con, 1,0)

# df_miss_SciName.to_csv('meta/mlcn_missed_checklist.csv')
df_miss_SciName = pd.read_csv('meta/mlcn_missed_checklist.csv', index_col=0)
df_miss_SciName

,ML Catalog Number,Scientific Name,Macaulay_public
0,35118661,Ornithion inerme,1
1,43334941,Thamnophilus nigrocinereus,1
2,43334951,Thamnophilus nigrocinereus,1
3,43334961,Thamnophilus nigrocinereus,1
4,43334971,Thamnophilus nigrocinereus,1
5,82069831,Puffinus yelkouan/mauretanicus,1
6,82069851,Puffinus yelkouan/mauretanicus,1
7,113563121,Empidonax difficilis/occidentalis,1
8,125398821,NaN,0
9,135237181,NaN,0


- 合併 df_meta與df_miss_meta

In [11]:
%time
df_meta_all = pd.read_csv('meta/meta_all.csv', index_col=0)

Wall time: 0 ns


In [17]:
%%time
# df_meta_all = (df_meta_all.iloc[:,2:]
#                .drop_duplicates(subset=['ML Catalog Number'], keep='first')
#                .sort_values(by=['ML Catalog Number'])
#                )
# df_meta_all.to_csv('meta/meta_all_mlcn_dropdup.csv')


Wall time: 8min 48s


In [66]:
%%time
# df_meta_all.iloc[:,1:].to_csv('meta/meta_all_mlcn_dropdup.csv')
# df_meta_all = df_meta_all .iloc[:,1:]

Wall time: 1min 5s


In [37]:
%%time
# df_meta_all = pd.read('meta/meta_all_mlcn_dropdup.csv', index_col=0)


Wall time: 2 ms


In [94]:
mlcns_meta = df_meta_all['ML Catalog Number'].values
mlcns = df_miss_SciName['ML Catalog Number'].values

idx_miss_meta = {}
for mlcn in mlcns:
    try: 
        idx_miss_meta[mlcn] = int(np.where(mlcns_meta == mlcn)[0])
    except:
        continue

print(idx_miss_meta)

df_miss_meta = pd.concat(
                        [df_meta_all.iloc[idx] for idx in idx_miss_meta.values() ],
                        axis=1
                        ).T

df_miss_meta
    

{113563121: 2660464, 135237181: 3461137, 250281361: 9997820, 250281371: 9997821}


,ML Catalog Number,Format,Scientific Name,Common Name,Background Species,Recordist,Date,Year,Month,Day,...,Asset Tags,Contributor Profile URL,Original Image Height,Original Image Width,Specimen Page URL,eBird Checklist URL,ebird Species URL,isInternalUser,mediaDownloadUrl,report_as
12101638,113563121,Photo,Empidonax difficilis/occidentalis,Pacific-slope/Cordilleran Flycatcher (Western ...,NaN,Chris Hayward,8/22/2018,2018.0,8.0,22.0,...,NaN,NaN,2974.0,3965.0,https://macaulaylibrary.org/asset/113563121,https://ebird.org/view/checklist/S48289202,NaN,False,https://cdn.download.ams.birds.cornell.edu/api...,wesfly
4654061,135237181,Photo,Phacellodomus rufifrons inornatus/castilloi,Rufous-fronted Thornbird (Plain),NaN,Jose Salguero,5/11/2009,2009.0,5.0,11.0,...,NaN,NaN,646.0,789.0,https://macaulaylibrary.org/asset/135237181,https://ebird.org/view/checklist/S22646676,https://ebird.org/species/ruftho1,False,https://cdn.download.ams.birds.cornell.edu/api...,ruftho1
12135821,250281361,Photo,Empidonax oberholseri,Dusky Flycatcher,NaN,Jerry Webb,7/17/2020,2020.0,7.0,17.0,...,NaN,https://ebird.org/profile/MzQ1MjM5,1507.0,1206.0,https://macaulaylibrary.org/asset/250281361,https://ebird.org/view/checklist/S71595125,https://ebird.org/species/dusfly,False,https://cdn.download.ams.birds.cornell.edu/api...,dusfly
12135820,250281371,Photo,Empidonax oberholseri,Dusky Flycatcher,NaN,Jerry Webb,7/17/2020,2020.0,7.0,17.0,...,NaN,https://ebird.org/profile/MzQ1MjM5,1375.0,1100.0,https://macaulaylibrary.org/asset/250281371,https://ebird.org/view/checklist/S71595125,https://ebird.org/species/dusfly,False,https://cdn.download.ams.birds.cornell.edu/api...,dusfly


In [142]:
# df_miss_tmp[df_miss_tmp['ML Catalog Number'] == 35118661]['Scientific Name'] = 'Ornithion inerme'
# df_miss_tmp[df_miss_tmp['ML Catalog Number'] == 35118661]['Scientific Name']
df_miss_tmp.iloc[0,3] = 'Ornithion inerme'


array([0], dtype=int64)

In [144]:

df_miss_tmp = df_miss_SciName[['ML Catalog Number','Macaulay_public']].merge(df_miss_meta, on='ML Catalog Number', how='left')
# for mlcn which 'Scientific Name' is na in metadata
mlcn_for_fill = df_miss_tmp[df_miss_tmp['Scientific Name'].isna()]['ML Catalog Number'].values
mlcn_for_fill

for mlcn in mlcn_for_fill:
    name = df_miss_SciName[df_miss_SciName['ML Catalog Number'] == mlcn]['Scientific Name'].values[0]
    idx = df_miss_tmp[df_miss_tmp['ML Catalog Number'] == mlcn]['Scientific Name'].index.values
    # print(mlcn, name)
    df_miss_tmp.loc[idx,'Scientific Name'] = name

df_miss_tmp

,ML Catalog Number,Macaulay_public,Format,Scientific Name,Common Name,Background Species,Recordist,Date,Year,Month,...,Asset Tags,Contributor Profile URL,Original Image Height,Original Image Width,Specimen Page URL,eBird Checklist URL,ebird Species URL,isInternalUser,mediaDownloadUrl,report_as
0,35118661,1,NaN,Ornithion inerme,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43334941,1,NaN,Thamnophilus nigrocinereus,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,43334951,1,NaN,Thamnophilus nigrocinereus,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,43334961,1,NaN,Thamnophilus nigrocinereus,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43334971,1,NaN,Thamnophilus nigrocinereus,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,82069831,1,NaN,Puffinus yelkouan/mauretanicus,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,82069851,1,NaN,Puffinus yelkouan/mauretanicus,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,113563121,1,Photo,Empidonax difficilis/occidentalis,Pacific-slope/Cordilleran Flycatcher (Western ...,NaN,Chris Hayward,8/22/2018,2018.0,8.0,...,NaN,NaN,2974.0,3965.0,https://macaulaylibrary.org/asset/113563121,https://ebird.org/view/checklist/S48289202,NaN,False,https://cdn.download.ams.birds.cornell.edu/api...,wesfly
8,125398821,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,135237181,0,Photo,Phacellodomus rufifrons inornatus/castilloi,Rufous-fronted Thornbird (Plain),NaN,Jose Salguero,5/11/2009,2009.0,5.0,...,NaN,NaN,646.0,789.0,https://macaulaylibrary.org/asset/135237181,https://ebird.org/view/checklist/S22646676,https://ebird.org/species/ruftho1,False,https://cdn.download.ams.birds.cornell.edu/api...,ruftho1


In [146]:
df_top100_meta =  pd.read_csv('meta/jpg_top100_meta.csv', index_col=0)
df_top100_meta = df_top100_meta[df_top100_meta['Scientific Name'].notnull()]

,ML Catalog Number,Dir_FName,Filename,Sci_N,Size,Scientific Name,Common Name,Background Species,Recordist,Date,...,Asset Tags,Contributor Profile URL,Original Image Height,Original Image Width,Specimen Page URL,eBird Checklist URL,ebird Species URL,isInternalUser,mediaDownloadUrl,report_as
0,22413261,Acanthisittidae/Acanthisitta_chloris_0_22413261,Acanthisitta_chloris_0_22413261,Acanthisitta_chloris_0,61.48,Acanthisitta chloris,Rifleman,NaN,Ric Else,12/24/2015,...,NaN,https://ebird.org/profile/NDQ0NzUz,800.0,572.0,https://macaulaylibrary.org/asset/22413261,https://ebird.org/view/checklist/S26454257,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
1,22413271,Acanthisittidae/Acanthisitta_chloris_0_22413271,Acanthisitta_chloris_0_22413271,Acanthisitta_chloris_0,68.64,Acanthisitta chloris,Rifleman,NaN,Ric Else,12/24/2015,...,NaN,https://ebird.org/profile/NDQ0NzUz,534.0,800.0,https://macaulaylibrary.org/asset/22413271,https://ebird.org/view/checklist/S26454257,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
2,30352591,Acanthisittidae/Acanthisitta_chloris_0_30352591,Acanthisitta_chloris_0_30352591,Acanthisitta_chloris_0,132.18,Acanthisitta chloris,Rifleman,NaN,Imogen Warren,1/10/2016,...,watermark,https://ebird.org/profile/MTgyODQ1,681.0,800.0,https://macaulaylibrary.org/asset/30352591,https://ebird.org/view/checklist/S30294988,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
3,40629251,Acanthisittidae/Acanthisitta_chloris_0_40629251,Acanthisitta_chloris_0_40629251,Acanthisitta_chloris_0,176.57,Acanthisitta chloris,Rifleman,NaN,Ryne Rutherford,3/24/2015,...,NaN,https://ebird.org/profile/MzA5ODM5,3000.0,4512.0,https://macaulaylibrary.org/asset/40629251,https://ebird.org/view/checklist/S22738170,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
4,41060091,Acanthisittidae/Acanthisitta_chloris_0_41060091,Acanthisitta_chloris_0_41060091,Acanthisitta_chloris_0,165.23,Acanthisitta chloris,Rifleman,NaN,Cathy Pasterczyk,11/17/2016,...,NaN,https://ebird.org/profile/NDM3MjA1,1114.0,1670.0,https://macaulaylibrary.org/asset/41060091,https://ebird.org/view/checklist/S32605669,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019963,222411491,Tyrannidae/Mionectes_macconnelli_0_222411491,Mionectes_macconnelli_0_222411491,Mionectes_macconnelli_0,144.18,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019964,222411541,Tyrannidae/Mionectes_macconnelli_0_222411541,Mionectes_macconnelli_0_222411541,Mionectes_macconnelli_0,126.77,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019965,222411551,Tyrannidae/Mionectes_macconnelli_0_222411551,Mionectes_macconnelli_0_222411551,Mionectes_macconnelli_0,146.52,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019966,205696561,Tyrannidae/Muscisaxicola_rufivertex_3_205696561,Muscisaxicola_rufivertex_3_205696561,Muscisaxicola_rufivertex_3,175.81,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
df_top100_meta = df_top100_meta[df_top100_meta['Scientific Name'].notnull()]
df_top100_meta

,ML Catalog Number,Dir_FName,Filename,Sci_N,Size,Scientific Name,Common Name,Background Species,Recordist,Date,...,Asset Tags,Contributor Profile URL,Original Image Height,Original Image Width,Specimen Page URL,eBird Checklist URL,ebird Species URL,isInternalUser,mediaDownloadUrl,report_as
0,22413261,Acanthisittidae/Acanthisitta_chloris_0_22413261,Acanthisitta_chloris_0_22413261,Acanthisitta_chloris_0,61.48,Acanthisitta chloris,Rifleman,NaN,Ric Else,12/24/2015,...,NaN,https://ebird.org/profile/NDQ0NzUz,800.0,572.0,https://macaulaylibrary.org/asset/22413261,https://ebird.org/view/checklist/S26454257,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
1,22413271,Acanthisittidae/Acanthisitta_chloris_0_22413271,Acanthisitta_chloris_0_22413271,Acanthisitta_chloris_0,68.64,Acanthisitta chloris,Rifleman,NaN,Ric Else,12/24/2015,...,NaN,https://ebird.org/profile/NDQ0NzUz,534.0,800.0,https://macaulaylibrary.org/asset/22413271,https://ebird.org/view/checklist/S26454257,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
2,30352591,Acanthisittidae/Acanthisitta_chloris_0_30352591,Acanthisitta_chloris_0_30352591,Acanthisitta_chloris_0,132.18,Acanthisitta chloris,Rifleman,NaN,Imogen Warren,1/10/2016,...,watermark,https://ebird.org/profile/MTgyODQ1,681.0,800.0,https://macaulaylibrary.org/asset/30352591,https://ebird.org/view/checklist/S30294988,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
3,40629251,Acanthisittidae/Acanthisitta_chloris_0_40629251,Acanthisitta_chloris_0_40629251,Acanthisitta_chloris_0,176.57,Acanthisitta chloris,Rifleman,NaN,Ryne Rutherford,3/24/2015,...,NaN,https://ebird.org/profile/MzA5ODM5,3000.0,4512.0,https://macaulaylibrary.org/asset/40629251,https://ebird.org/view/checklist/S22738170,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
4,41060091,Acanthisittidae/Acanthisitta_chloris_0_41060091,Acanthisitta_chloris_0_41060091,Acanthisitta_chloris_0,165.23,Acanthisitta chloris,Rifleman,NaN,Cathy Pasterczyk,11/17/2016,...,NaN,https://ebird.org/profile/NDM3MjA1,1114.0,1670.0,https://macaulaylibrary.org/asset/41060091,https://ebird.org/view/checklist/S32605669,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019919,219727771,Zosteropidae/Zosterornis_whiteheadi_0_219727771,Zosterornis_whiteheadi_0_219727771,Zosterornis_whiteheadi_0,310.95,Zosterornis whiteheadi,Chestnut-faced Babbler,NaN,Ron Hoff Dollyann Myers,1/15/2020,...,NaN,https://ebird.org/profile/NDQ2NDA4,1448.0,1728.0,https://macaulaylibrary.org/asset/219727771,https://ebird.org/view/checklist/S66463325,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1
1019920,219727781,Zosteropidae/Zosterornis_whiteheadi_0_219727781,Zosterornis_whiteheadi_0_219727781,Zosterornis_whiteheadi_0,349.39,Zosterornis whiteheadi,Chestnut-faced Babbler,NaN,Ron Hoff Dollyann Myers,1/15/2020,...,NaN,https://ebird.org/profile/NDQ2NDA4,1475.0,1761.0,https://macaulaylibrary.org/asset/219727781,https://ebird.org/view/checklist/S66463325,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1
1019921,219727791,Zosteropidae/Zosterornis_whiteheadi_0_219727791,Zosterornis_whiteheadi_0_219727791,Zosterornis_whiteheadi_0,368.17,Zosterornis whiteheadi,Chestnut-faced Babbler,NaN,Ron Hoff Dollyann Myers,1/15/2020,...,NaN,https://ebird.org/profile/NDQ2NDA4,1546.0,1728.0,https://macaulaylibrary.org/asset/219727791,https://ebird.org/view/checklist/S66463325,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1
1019922,219727801,Zosteropidae/Zosterornis_whiteheadi_0_219727801,Zosterornis_whiteheadi_0_219727801,Zosterornis_whiteheadi_0,324.02,Zosterornis whi

In [151]:
print(len(df_miss_tmp.columns))
print(len(df_top100_meta.columns))

58
60


In [161]:
df_jpg100 = pd.read_csv('meta/jpg_top100_drop_dup.txt',
                        usecols=[4,8],
                        names=['Size', 'Path'],
                        sep=' ',
                        header=None 
                        )

df_jpg100 = df_jpg100.query('~Path.str.contains("checkpoint")').reset_index(drop=True)
def get_family(s:pd.Series):
    return Path(s).parent.name
def get_basename(s:pd.Series):
    return Path(s).stem
def get_Dir_FName(s:pd.Series):
    return Path(s).parent.name + '/' + Path(s).stem

df_jpg100 = df_jpg100.query('~Path.str.contains("checkpoint")').reset_index(drop=True)
# df_jpg100['Parent_Dir'] = df_jpg100.Path.apply(get_family)
df_jpg100['Filename'] = df_jpg100.Path.apply(get_basename)
df_jpg100['Dir_FName'] = df_jpg100.Path.apply(get_Dir_FName)

df_jpg100['ML Catalog Number'] = df_jpg100.Dir_FName.apply(lambda x: x.split('_')[-1]).astype('uint32')
# df_jpg100['Sci_N'] = df_jpg100.Dir_FName.str.extract(r'([\w]+_[a-z]+_[\d]{1,3})')
# convert byte to kb
# df_jpg100['Size'] = df_jpg100.Size.apply(lambda x: round(x/1024,2))                     

df_jpg100 = df_jpg100[['ML Catalog Number','Dir_FName', 'Filename', ]]

df_jpg100 = df_jpg100.sort_values(by=['Dir_FName']).reset_index(drop=True)


In [157]:
df_jpg100

,ML Catalog Number,Dir_FName,Filename,Parent_Dir
0,114289801,Acanthisittidae/Acanthisitta_chloris_0_114289801,Acanthisitta_chloris_0_114289801,Acanthisittidae
1,114289821,Acanthisittidae/Acanthisitta_chloris_0_114289821,Acanthisitta_chloris_0_114289821,Acanthisittidae
2,114404941,Acanthisittidae/Acanthisitta_chloris_0_114404941,Acanthisitta_chloris_0_114404941,Acanthisittidae
3,114404951,Acanthisittidae/Acanthisitta_chloris_0_114404951,Acanthisitta_chloris_0_114404951,Acanthisittidae
4,115495451,Acanthisittidae/Acanthisitta_chloris_0_115495451,Acanthisitta_chloris_0_115495451,Acanthisittidae
...,...,...,...,...
896550,59630711,Zosteropidae/Zosterornis_whiteheadi_0_59630711,Zosterornis_whiteheadi_0_59630711,Zosteropidae
896551,59630721,Zosteropidae/Zosterornis_whiteheadi_0_59630721,Zosterornis_whiteheadi_0_59630721,Zosteropidae
896552,63554951,Zosteropidae/Zosterornis_whiteheadi_0_63554951,Zosterornis_whiteheadi_0_63554951,Zosteropidae
896553,79738531,Zosteropidae/Zosterornis_whiteheadi_0_79738531,Zosterornis_whiteheadi_0_79738531,Zosteropidae


In [166]:
df_top100_meta_ = pd.concat([df_top100_meta,
           df_jpg100.merge(df_miss_tmp, on='ML Catalog Number', how='inner')
           ]).reset_index(drop=True)

In [168]:
col= df_miss_tmp.columns
df_top100_meta_[col].to_csv('meta/jpg_top100_meta2.csv')

In [ ]:
# col= ['ML Catalog Number']
# df_miss_ = df_miss.merge(df_miss_SciName, left_on=col, right_on=col)
# df_miss_

In [380]:

# df_meta = pd.read_csv('meta/jpg_top100_meta.csv', index_col=0)
# df_meta 
# print(len(df_meta))
# print(len(df_jpg100))

# df_meta = pd.concat([df_meta, df_miss_meta])
# print(len(df_meta))
# print(len(df_jpg100))

# df_meta.to_csv('meta/jpg_top100_meta.csv')

---

## Checking meta data

In [ ]:
# df_meta = pd.read_csv('meta/jpg_top100_meta.csv', index_col=0)
# df_meta.info()

In [425]:
# Check Taxon fields is empty?
# df_meta[[ 'Class', 'Order', 'Family', 'Parent Species', 'eBird Species Code','Taxon Category']].isnull().apply(lambda x  : x.values.all())

Class                  True
Order                  True
Family                 True
Parent Species        False
eBird Species Code    False
Taxon Category        False
dtype: bool

### ~~修改meta data 欄位~~

In [206]:

# col_meta_ori = ['ML Catalog Number', 'Scientific Name', 'Common Name', 'Background Species', 'Recordist', 'Date', 'Year', 'Month', 'Day', 'Time', 
#               'Country', 'Country-State-County', 'State', 'County', 'Locality', 'Latitude', 'Longitude', 'Elevation (m)', 
#               'Age/Sex', 'Behaviors', 'Playback', 'Collected', 'Specimen ID', 'Home Archive Catalog Number',
#               'Recorder', 'Microphone', 'Accessory', 'Partner Institution', 'eBird Checklist ID', 'Unconfirmed?', 'Air Temp(ºC)', 'Water Temp(ºC)', 'Media notes', 'Observation Details', 
#               'Class', 'Order', 'Family', 'Parent Species', 'eBird Species Code', 'Taxon Category', 'Taxonomic Sort', 
#               'Recordist 2 Display Name', 'Recordist 2 Profile URL', 'Average Community Rating', 'Number of Ratings', 'Asset State', 'Asset Tags', 'Contributor Profile URL',
#               'Original Image Height', 'Original Image Width', 'Specimen Page URL', 'eBird Checklist URL', 'ebird Species URL', 'isInternalUser', 'mediaDownloadUrl', 'report_as']
# col_jpg100 = ['ML Catalog Number','Dir_FName','Filename','SciN','Size']
# col_meta = [*col_jpg100, *col_meta_ori[1:]]


-  ~~原ebird meta data欄位的類群資料全為空值 因此捨棄原'Family'欄位 採現有資料中的Family欄位~~

In [ ]:

# 原ebird meta data欄位的類群資料全為nan，因此捨棄原'Family'欄位，採現有資料中的Family欄位
# df_meta = pd.merge(
#                     df_meta.drop(columns=['Size','Family']),
#                     df_jpg100[['Family', *col_jpg100]],
#                     on=['ML Catalog Number','Filename']
#                     )[col_meta]

# df_meta.to_csv('meta/jpg_top100_meta.csv')
# df_meta = pd.read_csv('meta/jpg_top100_meta.csv', index_col=0)
# df_meta 


## ~~檢查'Sci_N'(學名_編碼) 與 meta資料欄位'Scientific Name'(學名)不一致的資料~~

In [ ]:
# df_top100 = pd.read_csv('meta/jpg_top100_meta.csv', index_col=0)
# df_top100

In [204]:
# df_top100.columns

# col = ['ML Catalog Number', 'Dir_FName', 'Filename', 'Sci_N', 'Scientific Name', 'Common Name', 'Specimen ID','Parent Species','eBird Species Code', 'Taxon Category']
# df_top100[col]

In [ ]:
# df_ = (df_top100.Sci_N.iloc[idx]
# .str.extract(r'([\w]+_[a-z]+)')
# .apply(lambda x :  x.str.replace('_', ' '))
# .set_axis(['Sci_'], axis='columns')
# )
# df_

## 檢查mlcn重複的資料

In [547]:
# del df_meta, df_jpg100

In [24]:
df_top100 = pd.read_csv('meta/jpg_top100_meta.csv', index_col=0)
df_top100

/home/esslab/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (29,30,31,58) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/esslab/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,ML Catalog Number,Dir_FName,Filename,Sci_N,Size,Scientific Name,Common Name,Background Species,Recordist,Date,...,Asset Tags,Contributor Profile URL,Original Image Height,Original Image Width,Specimen Page URL,eBird Checklist URL,ebird Species URL,isInternalUser,mediaDownloadUrl,report_as
0,22413261,Acanthisittidae/Acanthisitta_chloris_0_22413261,Acanthisitta_chloris_0_22413261,Acanthisitta_chloris_0,61.48,Acanthisitta chloris,Rifleman,NaN,Ric Else,12/24/2015,...,NaN,https://ebird.org/profile/NDQ0NzUz,800.0,572.0,https://macaulaylibrary.org/asset/22413261,https://ebird.org/view/checklist/S26454257,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
1,22413271,Acanthisittidae/Acanthisitta_chloris_0_22413271,Acanthisitta_chloris_0_22413271,Acanthisitta_chloris_0,68.64,Acanthisitta chloris,Rifleman,NaN,Ric Else,12/24/2015,...,NaN,https://ebird.org/profile/NDQ0NzUz,534.0,800.0,https://macaulaylibrary.org/asset/22413271,https://ebird.org/view/checklist/S26454257,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
2,30352591,Acanthisittidae/Acanthisitta_chloris_0_30352591,Acanthisitta_chloris_0_30352591,Acanthisitta_chloris_0,132.18,Acanthisitta chloris,Rifleman,NaN,Imogen Warren,1/10/2016,...,watermark,https://ebird.org/profile/MTgyODQ1,681.0,800.0,https://macaulaylibrary.org/asset/30352591,https://ebird.org/view/checklist/S30294988,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
3,40629251,Acanthisittidae/Acanthisitta_chloris_0_40629251,Acanthisitta_chloris_0_40629251,Acanthisitta_chloris_0,176.57,Acanthisitta chloris,Rifleman,NaN,Ryne Rutherford,3/24/2015,...,NaN,https://ebird.org/profile/MzA5ODM5,3000.0,4512.0,https://macaulaylibrary.org/asset/40629251,https://ebird.org/view/checklist/S22738170,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
4,41060091,Acanthisittidae/Acanthisitta_chloris_0_41060091,Acanthisitta_chloris_0_41060091,Acanthisitta_chloris_0,165.23,Acanthisitta chloris,Rifleman,NaN,Cathy Pasterczyk,11/17/2016,...,NaN,https://ebird.org/profile/NDM3MjA1,1114.0,1670.0,https://macaulaylibrary.org/asset/41060091,https://ebird.org/view/checklist/S32605669,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019963,222411491,Tyrannidae/Mionectes_macconnelli_0_222411491,Mionectes_macconnelli_0_222411491,Mionectes_macconnelli_0,144.18,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019964,222411541,Tyrannidae/Mionectes_macconnelli_0_222411541,Mionectes_macconnelli_0_222411541,Mionectes_macconnelli_0,126.77,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019965,222411551,Tyrannidae/Mionectes_macconnelli_0_222411551,Mionectes_macconnelli_0_222411551,Mionectes_macconnelli_0,146.52,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019966,205696561,Tyrannidae/Muscisaxicola_rufivertex_3_205696561,Muscisaxicola_rufivertex_3_205696561,Muscisaxicola_rufivertex_3,175.81,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_top100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1019968 entries, 0 to 1019967
Data columns (total 60 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   ML Catalog Number            1019968 non-null  int64  
 1   Dir_FName                    1019968 non-null  object 
 2   Filename                     1019968 non-null  object 
 3   Sci_N                        1019968 non-null  object 
 4   Size                         1019968 non-null  float64
 5   Scientific Name              1019924 non-null  object 
 6   Common Name                  1019924 non-null  object 
 7   Background Species           2427 non-null     object 
 8   Recordist                    1019874 non-null  object 
 9   Date                         1006617 non-null  object 
 10  Year                         1006617 non-null  float64
 11  Month                        1006617 non-null  float64
 12  Day                          1006617 non-n

In [40]:
# 抓取Scientific Name為NaN的資料，用Sci_N欄位填補
idx = df_top100[df_top100['Scientific Name'].isnull()].index.values
print(len(idx))

df_top100.loc[idx,'Scientific Name'] = (df_top100.Sci_N.iloc[idx]
                                        .str.extract(r'([\w]+_[a-z]+)')
                                        .apply(lambda x :  x.str.replace('_', ' '))
                                        ).values
df_top100.loc[idx] 


44


,ML Catalog Number,Dir_FName,Filename,Sci_N,Size,Scientific Name,Common Name,Background Species,Recordist,Date,...,Asset Tags,Contributor Profile URL,Original Image Height,Original Image Width,Specimen Page URL,eBird Checklist URL,ebird Species URL,isInternalUser,mediaDownloadUrl,report_as
1019924,236857561,Alaudidae/Alaudala_cheleensis_0_236857561,Alaudala_cheleensis_0_236857561,Alaudala_cheleensis_0,46.20,Alaudala cheleensis,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019925,135237181,Furnariidae/Phacellodomus_rufifrons_1_135237181,Phacellodomus_rufifrons_1_135237181,Phacellodomus_rufifrons_1,116.29,Phacellodomus rufifrons,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019926,125398821,Furnariidae/Synallaxis_courseni_0_125398821,Synallaxis_courseni_0_125398821,Synallaxis_courseni_0,443.20,Synallaxis courseni,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019927,204080901,Furnariidae/Syndactyla_striata_0_204080901,Syndactyla_striata_0_204080901,Syndactyla_striata_0,136.33,Syndactyla striata,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019928,204080911,Furnariidae/Syndactyla_striata_0_204080911,Syndactyla_striata_0_204080911,Syndactyla_striata_0,147.83,Syndactyla striata,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019929,255624301,Phasianidae/Francolinus_francolinus_2_255624301,Francolinus_francolinus_2_255624301,Francolinus_francolinus_2,210.20,Francolinus francolinus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019930,82069831,Procellariidae/Puffinus_mauretanicus_1_82069831,Puffinus_mauretanicus_1_82069831,Puffinus_mauretanicus_1,90.63,Puffinus mauretanicus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019931,82069851,Procellariidae/Puffinus_mauretanicus_1_82069851,Puffinus_mauretanicus_1_82069851,Puffinus_mauretanicus_1,112.28,Puffinus mauretanicus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019932,82069831,Procellariidae/Puffinus_puffinus_7_82069831,Puffinus_puffinus_7_82069831,Puffinus_puffinus_7,90.63,Puffinus puffinus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019933,82069851,Procellariidae/Puffinus_puffinus_7_82069851,Puffinus_puffinus_7_82069851,Puffinus_puffinus_7,112.28,Puffinus puffinus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### All data with mlcn duplicated

In [41]:
mlcn_dup  = (df_top100[df_top100.duplicated(subset=['ML Catalog Number'],keep=False)].sort_values(by='ML Catalog Number')
                 ['ML Catalog Number'].unique()
)
mlcn_dup = pd.DataFrame(mlcn_dup, columns=['ML Catalog Number'])

df_top100_dup = df_top100.loc[:,['ML Catalog Number','Dir_FName','Sci_N', 'Family', 'Scientific Name']].merge(mlcn_dup) 

print(f'Mlcn duplicated :              {len(mlcn_dup):,d}')
print(f'Data size of Mlcn duplicated : {len(df_top100_dup):,d}')


Mlcn duplicated :              114,798
Data size of Mlcn duplicated : 238,211


In [42]:
df_top100_dup.to_csv('meta/mlcn_duplicated.csv')

- With Same Scientific Name

In [43]:
mask = df_top100_dup.duplicated(subset=['Scientific Name', 'ML Catalog Number'], keep=False)
df_top100_dup_sp_same = df_top100_dup[mask]
mlcn_dup_sp_same = df_top100_dup_sp_same['ML Catalog Number'].unique()
print(f'\tDuplicated Mlcns with same "Scientific Name":             {len(mlcn_dup_sp_same):,d}')
print(f'\tData size of Mlcn duplicated with same "Scientific Name": {len(df_top100_dup_sp_same):,d}')


	Duplicated Mlcns with same "Scientific Name":             114,793
	Data size of Mlcn duplicated with same "Scientific Name": 238,198


- With Different Scientific Name

In [44]:
mlcn_dup_sp_diff = df_top100_dup[~mask]['ML Catalog Number'].unique()
mlcn_dup_sp_diff = pd.DataFrame(mlcn_dup_sp_diff, columns=['ML Catalog Number']).sort_values(by=['ML Catalog Number'])
print(len(mlcn_dup_sp_diff))
mlcn_dup_sp_diff

6


,ML Catalog Number
2,82069831
3,82069851
1,113563121
0,135237181
4,250281361
5,250281371


In [45]:
df_top100_dup_sp_diff = df_top100_dup.merge(mlcn_dup_sp_diff).sort_values(by=['ML Catalog Number','Sci_N'])
df_top100_dup_sp_diff.to_csv('meta/mlcn_duplicated_sp_diff.csv')
df_top100_dup_sp_diff

,ML Catalog Number,Dir_FName,Sci_N,Family,Scientific Name
5,82069831,Procellariidae/Puffinus_mauretanicus_1_82069831,Puffinus_mauretanicus_1,Procellariidae,Puffinus mauretanicus
6,82069831,Procellariidae/Puffinus_puffinus_7_82069831,Puffinus_puffinus_7,Procellariidae,Puffinus puffinus
7,82069831,Procellariidae/Puffinus_yelkouan_1_82069831,Puffinus_yelkouan_1,Procellariidae,Puffinus yelkouan
8,82069851,Procellariidae/Puffinus_mauretanicus_1_82069851,Puffinus_mauretanicus_1,Procellariidae,Puffinus mauretanicus
9,82069851,Procellariidae/Puffinus_puffinus_7_82069851,Puffinus_puffinus_7,Procellariidae,Puffinus puffinus
10,82069851,Procellariidae/Puffinus_yelkouan_1_82069851,Puffinus_yelkouan_1,Procellariidae,Puffinus yelkouan
2,113563121,Tyrannidae/Empidonax_difficilis_2_113563121,Empidonax_difficilis_2,Tyrannidae,Empidonax difficilis/occidentalis
3,113563121,Tyrannidae/Empidonax_occidentalis_2_113563121,Empidonax_occidentalis_2,Tyrannidae,Empidonax difficilis/occidentalis
4,113563121,Tyrannidae/Empidonax_traillii_3_113563121,Empidonax_traillii_3,Tyrannidae,Empidonax traillii
0,135237181,Furnariidae/Phacellodomus_inornatus_0_135237181,Phacellodomus_inornatus_0,Furnariidae,Phacellodomus rufifrons inornatus/castilloi


In [8]:
print(f'Mlcn duplicated')
print(f'All :  {len(mlcn_dup):,d}')
print(f'\tSame "Scientific Name" :      {len(mlcn_dup_sp_same):7,d}')
print(f'\tDifferent "Scientific Name" : {len(mlcn_dup_sp_diff):7,d}')
print('-'*50)
print('Data size of Mlcn duplicated :')
print(f'All :  {len(df_top100_dup):,d}')
print(f'\tSame "Scientific Name" :      {len(df_top100_dup_sp_same):7,d}')
print(f'\tDifferent "Scientific Name" : {len(df_top100_dup_sp_diff):7,d}')

Mlcn duplicated
All :  114,798
	Same "Scientific Name" :      114,793
	Different "Scientific Name" :       6
--------------------------------------------------
Data size of Mlcn duplicated :
All :  238,211
	Same "Scientific Name" :      238,198
	Different "Scientific Name" :      15


### ~~匯出重複的資料清單~~
- 所有資料
    - 不同學名
    - 同一學名

In [9]:
import shutil

dir_top100 = Path('downloaded/jpg_top100')
dir_save = Path('Duplicated')

- 不同學名

In [10]:
# dir_save = Path('Duplicated/sp_diff')
# dir_save.mkdir(parents=True, exist_ok=True) 

# for i,  (idx, rows) in enumerate(df_top100_dup_sp_diff.iterrows()):
    
#     mlcn, dir_fname, sci_n, family, sp = rows
#     fname = Path(dir_fname).stem
#     print(i, idx, dir_fname, end='\r')
    
#     load_path = dir_top100.joinpath(dir_fname + '.jpg') 
#     save_path = dir_save.joinpath(fname  + '.jpg') 
#     shutil.copyfile(load_path, save_path)

- 同學名

In [ ]:
# dir_save = Path('Duplicated/sp_same')

# for i,  (idx, rows) in enumerate(df_top100_dup_sp_same.iterrows()):
    
#     mlcn, dir_fname, sci_n, family, sp  = rows
#     fname = Path(dir_fname).stem
#     print(i, idx, dir_fname, end='\r')
    
#     dir_save_ = dir_save.joinpath(family)
#     if not dir_save_.exists():
#         dir_save_.mkdir(parents=True, exist_ok=True) 
    
#     load_path = dir_top100.joinpath(dir_fname + '.jpg') 
#     save_path = dir_save_.joinpath(fname  + '.jpg') 
    
#     shutil.copyfile(load_path, save_path)

## 刪除mlcn重複的資料

### mlcn重複、但學名不一致的資料 
- 統一學名後刪除
- 依據mlcn值直接到網站查詢，目前使用的學名  https://macaulaylibrary.org/asset/'mlcn'
- 資料填寫至'meta/mlcn_duplicated_checklist.csv'
- 以目前網站資料為準，刪除重複(保留第一筆)

In [172]:
df_checklist = pd.read_csv('meta/mlcn_duplicated_checklist.csv', index_col=0)
print(df_checklist.columns)
df_checklist

Index(['ML Catalog Number', 'Scientific Name', 'Macaulay_public'], dtype='object')


,ML Catalog Number,Scientific Name,Macaulay_public
0,82069831,Puffinus yelkouan/mauretanicus,1
1,82069851,Puffinus yelkouan/mauretanicus,1
2,113563121,Empidonax difficilis/occidentalis,1
3,135237181,NaN,0
4,250281361,Empidonax oberholseri,1
5,250281371,Empidonax oberholseri,1


In [101]:
temp = (df_top100_dup_sp_diff[['ML Catalog Number', 'Scientific Name']]
        .merge(df_checklist, 
                left_on=['ML Catalog Number'], right_on=['ML Catalog Number'],  
                how='outer', suffixes=('_meta', '')
                )
        #  .dropna()
        .drop_duplicates(subset=['ML Catalog Number'], keep='first')
        )
temp

,ML Catalog Number,Scientific Name_meta,Scientific Name,Macaulay_public
0,82069831,Puffinus mauretanicus,Puffinus yelkouan/mauretanicus,1
3,82069851,Puffinus mauretanicus,Puffinus yelkouan/mauretanicus,1
6,113563121,Empidonax difficilis/occidentalis,Empidonax difficilis/occidentalis,1
9,135237181,Phacellodomus rufifrons inornatus/castilloi,NaN,0
11,250281361,Empidonax oberholseri,Empidonax oberholseri,1
13,250281371,Empidonax oberholseri,Empidonax oberholseri,1


In [124]:
df_dup_droped_diff = (df_top100_dup_sp_diff.iloc[:,:2]
                     .merge(temp[['ML Catalog Number', 'Scientific Name', 'Macaulay_public']], 
                            left_on=['ML Catalog Number'], right_on=['ML Catalog Number'])
                     .drop_duplicates(subset=['ML Catalog Number'], keep='first')
                     )
df_dup_droped_diff

,ML Catalog Number,Dir_FName,Scientific Name,Macaulay_public
0,82069831,Procellariidae/Puffinus_mauretanicus_1_82069831,Puffinus yelkouan/mauretanicus,1
3,82069851,Procellariidae/Puffinus_mauretanicus_1_82069851,Puffinus yelkouan/mauretanicus,1
6,113563121,Tyrannidae/Empidonax_difficilis_2_113563121,Empidonax difficilis/occidentalis,1
9,135237181,Furnariidae/Phacellodomus_inornatus_0_135237181,NaN,0
11,250281361,Tyrannidae/Empidonax_oberholseri_1_250281361,Empidonax oberholseri,1
13,250281371,Tyrannidae/Empidonax_oberholseri_1_250281371,Empidonax oberholseri,1


### mlcn重複、但學名相同的資料 

In [125]:
df_dup_droped_same =  (df_top100_dup_sp_same[['ML Catalog Number','Dir_FName', 'Scientific Name']]
                   .drop_duplicates(subset=['ML Catalog Number'], keep='first')
                   )
df_dup_droped_same['Macaulay_public'] = 1
df_dup_droped_same

,ML Catalog Number,Dir_FName,Scientific Name,Macaulay_public
0,22413261,Acanthisittidae/Acanthisitta_chloris_0_22413261,Acanthisitta chloris,1
2,22413271,Acanthisittidae/Acanthisitta_chloris_0_22413271,Acanthisitta chloris,1
4,30352591,Acanthisittidae/Acanthisitta_chloris_0_30352591,Acanthisitta chloris,1
6,40629251,Acanthisittidae/Acanthisitta_chloris_0_40629251,Acanthisitta chloris,1
8,41060091,Acanthisittidae/Acanthisitta_chloris_0_41060091,Acanthisitta chloris,1
...,...,...,...,...
238191,213688961,Zosteropidae/Zosterops_virens_0_213688961,Zosterops virens capensis,1
238193,213688971,Zosteropidae/Zosterops_virens_0_213688971,Zosterops virens capensis,1
238195,213689111,Zosteropidae/Zosterops_virens_0_213689111,Zosterops virens capensis,1
238197,215333921,Zosteropidae/Zosterops_virens_0_215333921,Zosterops virens virens,1


In [127]:
df_dup_droped =pd.concat([df_dup_droped_same, df_dup_droped_diff])
df_dup_droped

,ML Catalog Number,Dir_FName,Scientific Name,Macaulay_public
0,22413261,Acanthisittidae/Acanthisitta_chloris_0_22413261,Acanthisitta chloris,1
2,22413271,Acanthisittidae/Acanthisitta_chloris_0_22413271,Acanthisitta chloris,1
4,30352591,Acanthisittidae/Acanthisitta_chloris_0_30352591,Acanthisitta chloris,1
6,40629251,Acanthisittidae/Acanthisitta_chloris_0_40629251,Acanthisitta chloris,1
8,41060091,Acanthisittidae/Acanthisitta_chloris_0_41060091,Acanthisitta chloris,1
...,...,...,...,...
3,82069851,Procellariidae/Puffinus_mauretanicus_1_82069851,Puffinus yelkouan/mauretanicus,1
6,113563121,Tyrannidae/Empidonax_difficilis_2_113563121,Empidonax difficilis/occidentalis,1
9,135237181,Furnariidae/Phacellodomus_inornatus_0_135237181,NaN,0
11,250281361,Tyrannidae/Empidonax_oberholseri_1_250281361,Empidonax oberholseri,1


In [149]:
print(len(df_top100_dup))
col = ['ML Catalog Number','Dir_FName']
# tmp = (df_top100_dup[col].merge(df_dup_droped,
#                     left_on=col, right_on=col))
print(len(df_dup_droped))

df_dup_delete_list = (pd.concat([df_top100_dup[['ML Catalog Number', 'Dir_FName', 'Scientific Name']], df_dup_droped])
                      .drop_duplicates(subset=['ML Catalog Number','Dir_FName'], keep=False)
                      ).iloc[:,:-1]

print(len(df_dup_delete_list))
# assert len(df_top100_dup) - len(df_dup_droped) == len(df_dup_delete_list)
df_dup_delete_list


238211
114799
123413


,ML Catalog Number,Dir_FName,Scientific Name
1,22413261,Fringillidae/Chloris_chloris_6_22413261,Acanthisitta chloris
3,22413271,Fringillidae/Chloris_chloris_6_22413271,Acanthisitta chloris
5,30352591,Fringillidae/Chloris_chloris_6_30352591,Acanthisitta chloris
7,40629251,Fringillidae/Chloris_chloris_6_40629251,Acanthisitta chloris
9,41060091,Fringillidae/Chloris_chloris_6_41060091,Acanthisitta chloris
...,...,...,...
238203,82069831,Procellariidae/Puffinus_yelkouan_1_82069831,Puffinus yelkouan
238205,82069851,Procellariidae/Puffinus_puffinus_7_82069851,Puffinus puffinus
238206,82069851,Procellariidae/Puffinus_yelkouan_1_82069851,Puffinus yelkouan
238208,250281361,Tyrannidae/Empidonax_wrightii_1_250281361,Empidonax wrightii


In [150]:
df_dup_delete_list.to_csv('meta/dup_delete_list.csv')

- 刪除mlcn重複的檔案

In [5]:
dir_top100 = Path('downloaded/jpg_top100')

df_dup_delete = pd.read_csv('meta/dup_delete_list.csv', index_col=0)
df_dup_delete

,ML Catalog Number,Dir_FName,Scientific Name
1,22413261,Fringillidae/Chloris_chloris_6_22413261,Acanthisitta chloris
3,22413271,Fringillidae/Chloris_chloris_6_22413271,Acanthisitta chloris
5,30352591,Fringillidae/Chloris_chloris_6_30352591,Acanthisitta chloris
7,40629251,Fringillidae/Chloris_chloris_6_40629251,Acanthisitta chloris
9,41060091,Fringillidae/Chloris_chloris_6_41060091,Acanthisitta chloris
...,...,...,...
238203,82069831,Procellariidae/Puffinus_yelkouan_1_82069831,Puffinus yelkouan
238205,82069851,Procellariidae/Puffinus_puffinus_7_82069851,Puffinus puffinus
238206,82069851,Procellariidae/Puffinus_yelkouan_1_82069851,Puffinus yelkouan
238208,250281361,Tyrannidae/Empidonax_wrightii_1_250281361,Empidonax wrightii


In [11]:
for i,  (idx, rows) in enumerate(df_dup_delete.iterrows()):
    
    mlcn, dir_fname, sp = rows
    # fname = Path(dir_fname).stem
    print(f'{i:6,d}, {dir_fname:<30s}\t\t\t\t', end='\r')
    
    file_path = dir_top100.joinpath(dir_fname + '.jpg') 
    
    if file_path.exists():
        file_path.unlink()
        print(f'{i:6,d}, {dir_fname:30s} unlinked', end='\r')


# 刪除重複mlcn後再次建立jpg_top100目錄內meta檔
- meta data檔來源為ebird資料

#### ebird metat data 資料儲存方式
- ebird meta data位於downloaded/csv內
- 檔案目錄格式為 '科/學名_流水號.csv'
- 每個csv檔內容為該'學名_流水號'的所有影像的完整meta data

#### 處理流程
- 讀取資料 'meta/jpg_top100_drop_dup.txt'
- 與jpg_top100目錄建立的meta檔合併


- 讀取資料 'meta/jpg_top100_drop_dup.txt'

In [199]:
# from distributed import Client
# client = Client()
# import modin.pandas as pd
df_jpg100 = pd.read_csv('meta/jpg_top100_drop_dup.txt',
                        usecols=[4,8],
                        names=['Size', 'Path'],
                        sep=' ',
                        header=None 
                        )
# df_jpg100 = df_jpg100.query('~Path.str.contains("checkpoint")').reset_index(drop=True)
df_jpg100

,Size,Path
0,162891,downloaded/jpg_top100/Acanthisittidae/Acanthis...
1,296210,downloaded/jpg_top100/Acanthisittidae/Acanthis...
2,124934,downloaded/jpg_top100/Acanthisittidae/Acanthis...
3,294646,downloaded/jpg_top100/Acanthisittidae/Xenicus_...
4,230262,downloaded/jpg_top100/Acanthisittidae/Xenicus_...
...,...,...
896553,211764,downloaded/jpg_top100/Zosteropidae/Zosterops_m...
896554,109798,downloaded/jpg_top100/Zosteropidae/Zosterops_l...
896555,206092,downloaded/jpg_top100/Zosteropidae/Zosterops_l...
896556,165973,downloaded/jpg_top100/Zosteropidae/Zosterops_s...


In [200]:
# df_jpg100.query('Path.str.contains("checkpoint")').reset_index(drop=True)
# path_delete = df_jpg100.query('Path.str.contains("checkpoint")').reset_index(drop=True)['Path'].values
# print(path_delete)

# for path in path_delete:
#     Path(path).unlink()

In [201]:

# df_jpg100 = df_jpg100.query('~Path.str.contains("checkpoint")')
df_jpg100 = df_jpg100[~df_jpg100['Path'].str.contains("checkpoint")]
df_jpg100 

,Size,Path
0,162891,downloaded/jpg_top100/Acanthisittidae/Acanthis...
1,296210,downloaded/jpg_top100/Acanthisittidae/Acanthis...
2,124934,downloaded/jpg_top100/Acanthisittidae/Acanthis...
3,294646,downloaded/jpg_top100/Acanthisittidae/Xenicus_...
4,230262,downloaded/jpg_top100/Acanthisittidae/Xenicus_...
...,...,...
896553,211764,downloaded/jpg_top100/Zosteropidae/Zosterops_m...
896554,109798,downloaded/jpg_top100/Zosteropidae/Zosterops_l...
896555,206092,downloaded/jpg_top100/Zosteropidae/Zosterops_l...
896556,165973,downloaded/jpg_top100/Zosteropidae/Zosterops_s...


In [202]:
def get_family(s:pd.Series):
    return Path(s).parent.name
def get_basename(s:pd.Series):
    return Path(s).stem
def get_Dir_FName(s:pd.Series):
    return Path(s).parent.name + '/' + Path(s).stem


df_jpg100['Parent_Dir'] = df_jpg100.Path.apply(get_family)
# df_jpg100['Filename'] = df_jpg100.Path.apply(get_basename)
df_jpg100['Dir_FName'] = df_jpg100.Path.apply(get_Dir_FName)

df_jpg100['ML Catalog Number'] = df_jpg100.Dir_FName.apply(lambda x: x.split('_')[-1]).astype('uint32')
df_jpg100['Sci_N'] = df_jpg100.Dir_FName.str.extract(r'([\w]+_[a-z]+_[\d]{1,3})')
# convert byte to kb
df_jpg100['Size'] = df_jpg100.Size.apply(lambda x: round(x/1024,2))                     

df_jpg100 = df_jpg100[['ML Catalog Number','Dir_FName', 'Parent_Dir','Size', 'Sci_N']]

df_jpg100 = df_jpg100.sort_values(by=['Dir_FName']).reset_index(drop=True)

df_jpg100 

,ML Catalog Number,Dir_FName,Parent_Dir,Size,Sci_N
0,114289801,Acanthisittidae/Acanthisitta_chloris_0_114289801,Acanthisittidae,287.80,Acanthisitta_chloris_0
1,114289821,Acanthisittidae/Acanthisitta_chloris_0_114289821,Acanthisittidae,205.43,Acanthisitta_chloris_0
2,114404941,Acanthisittidae/Acanthisitta_chloris_0_114404941,Acanthisittidae,335.58,Acanthisitta_chloris_0
3,114404951,Acanthisittidae/Acanthisitta_chloris_0_114404951,Acanthisittidae,261.37,Acanthisitta_chloris_0
4,115495451,Acanthisittidae/Acanthisitta_chloris_0_115495451,Acanthisittidae,97.31,Acanthisitta_chloris_0
...,...,...,...,...,...
896550,59630711,Zosteropidae/Zosterornis_whiteheadi_0_59630711,Zosteropidae,211.91,Zosterornis_whiteheadi_0
896551,59630721,Zosteropidae/Zosterornis_whiteheadi_0_59630721,Zosteropidae,194.77,Zosterornis_whiteheadi_0
896552,63554951,Zosteropidae/Zosterornis_whiteheadi_0_63554951,Zosteropidae,254.81,Zosterornis_whiteheadi_0
896553,79738531,Zosteropidae/Zosterornis_whiteheadi_0_79738531,Zosteropidae,251.04,Zosterornis_whiteheadi_0


- 與jpg_top100目錄建立的meta檔合併

In [203]:
df_meta_  = pd.read_csv('meta/jpg_top100_meta2.csv', index_col=0)

In [204]:
df_meta_ = df_meta_.sort_values(by=['ML Catalog Number']).reset_index(drop=True)
mlcn_meta = df_meta_['ML Catalog Number'].values
df_meta_

,ML Catalog Number,Macaulay_public,Format,Scientific Name,Common Name,Background Species,Recordist,Date,Year,Month,...,Asset Tags,Contributor Profile URL,Original Image Height,Original Image Width,Specimen Page URL,eBird Checklist URL,ebird Species URL,isInternalUser,mediaDownloadUrl,report_as
0,700650,NaN,NaN,Cathartes aura jota,Turkey Vulture (South Temperate),NaN,Laurent Demongin,NaN,NaN,NaN,...,archive,https://ebird.org/profile/NTAxNTM3,1536.0,2048.0,https://macaulaylibrary.org/asset/700650,NaN,https://ebird.org/species/turvul,False,https://cdn.download.ams.birds.cornell.edu/api...,turvul
1,700658,NaN,NaN,Momotus coeruliceps,Blue-capped Motmot,NaN,José Alberto Lobato García,NaN,NaN,NaN,...,archive,https://ebird.org/profile/MjE0MTc3,448.0,286.0,https://macaulaylibrary.org/asset/700658,NaN,https://ebird.org/species/bucmot1,False,https://cdn.download.ams.birds.cornell.edu/api...,bucmot1
2,700666,NaN,NaN,Aptenodytes patagonicus,King Penguin,NaN,Laurent Demongin,NaN,NaN,NaN,...,archive,https://ebird.org/profile/NTAxNTM3,2176.0,3264.0,https://macaulaylibrary.org/asset/700666,NaN,https://ebird.org/species/kinpen1,False,https://cdn.download.ams.birds.cornell.edu/api...,kinpen1
3,700913,NaN,NaN,Anas capensis,Cape Teal,NaN,Megan Perkins,NaN,NaN,NaN,...,archive,NaN,575.0,850.0,https://macaulaylibrary.org/asset/700913,NaN,https://ebird.org/species/captea1,False,https://cdn.download.ams.birds.cornell.edu/api...,captea1
4,700917,NaN,NaN,Palmeria dolei,Akohekohe,NaN,Valerie Gebert,NaN,NaN,NaN,...,archive,https://ebird.org/profile/MTA1NDgy,1111.0,1631.0,https://macaulaylibrary.org/asset/700917,NaN,https://ebird.org/species/crehon,False,https://cdn.download.ams.birds.cornell.edu/api...,crehon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019957,264493941,NaN,NaN,Setophaga pensylvanica,Chestnut-sided Warbler,NaN,Teresa Noel,9/18/2020,2020.0,9.0,...,NaN,https://ebird.org/profile/MzA3MjEw,803.0,1072.0,https://macaulaylibrary.org/asset/264493941,https://ebird.org/view/checklist/S73736010,https://ebird.org/species/chswar,False,https://cdn.download.ams.birds.cornell.edu/api...,chswar
1019958,264493951,NaN,NaN,Setophaga pensylvanica,Chestnut-sided Warbler,NaN,Teresa Noel,9/18/2020,2020.0,9.0,...,NaN,https://ebird.org/profile/MzA3MjEw,736.0,981.0,https://macaulaylibrary.org/asset/264493951,https://ebird.org/view/checklist/S73736010,https://ebird.org/species/chswar,False,https://cdn.download.ams.birds.cornell.edu/api...,chswar
1019959,264493961,NaN,NaN,Setophaga pensylvanica,Chestnut-sided Warbler,NaN,Teresa Noel,9/18/2020,2020.0,9.0,...,NaN,https://ebird.org/profile/MzA3MjEw,783.0,1044.0,https://macaulaylibrary.org/asset/264493961,https://ebird.org/view/checklist/S73736010,https://ebird.org/species/chswar,False,https://cdn.download.ams.birds.cornell.edu/api...,chswar
1019960,264493971,NaN,NaN,Setophaga pensylvanica,Chestnut-sided Warbler,NaN,Teresa Noel,9/18/2020,2020.0,9.0,...,NaN,https://ebird.org/profile/MzA3MjEw,780.0,1040.0,https://macaulaylibrary.org/asset/264493971,https://ebird.org/view/checklist/S73736010,https://ebird.org/species/chswar,False,https://cdn.download.ams.birds.cornell.edu/api...,chswar


In [217]:
df_jpg100
mlcn_jpg100 = df_jpg100['ML Catalog Number'].values
df_jpg100

,ML Catalog Number,Dir_FName,Parent_Dir,Size,Sci_N
0,114289801,Acanthisittidae/Acanthisitta_chloris_0_114289801,Acanthisittidae,287.80,Acanthisitta_chloris_0
1,114289821,Acanthisittidae/Acanthisitta_chloris_0_114289821,Acanthisittidae,205.43,Acanthisitta_chloris_0
2,114404941,Acanthisittidae/Acanthisitta_chloris_0_114404941,Acanthisittidae,335.58,Acanthisitta_chloris_0
3,114404951,Acanthisittidae/Acanthisitta_chloris_0_114404951,Acanthisittidae,261.37,Acanthisitta_chloris_0
4,115495451,Acanthisittidae/Acanthisitta_chloris_0_115495451,Acanthisittidae,97.31,Acanthisitta_chloris_0
...,...,...,...,...,...
896550,59630711,Zosteropidae/Zosterornis_whiteheadi_0_59630711,Zosteropidae,211.91,Zosterornis_whiteheadi_0
896551,59630721,Zosteropidae/Zosterornis_whiteheadi_0_59630721,Zosteropidae,194.77,Zosterornis_whiteheadi_0
896552,63554951,Zosteropidae/Zosterornis_whiteheadi_0_63554951,Zosteropidae,254.81,Zosterornis_whiteheadi_0
896553,79738531,Zosteropidae/Zosterornis_whiteheadi_0_79738531,Zosteropidae,251.04,Zosterornis_whiteheadi_0


In [216]:
df_meta_  = df_meta_.drop_duplicates(subset=['ML Catalog Number'], keep='first')
df_meta_

,ML Catalog Number,Macaulay_public,Format,Scientific Name,Common Name,Background Species,Recordist,Date,Year,Month,...,Asset Tags,Contributor Profile URL,Original Image Height,Original Image Width,Specimen Page URL,eBird Checklist URL,ebird Species URL,isInternalUser,mediaDownloadUrl,report_as
0,700650,NaN,NaN,Cathartes aura jota,Turkey Vulture (South Temperate),NaN,Laurent Demongin,NaN,NaN,NaN,...,archive,https://ebird.org/profile/NTAxNTM3,1536.0,2048.0,https://macaulaylibrary.org/asset/700650,NaN,https://ebird.org/species/turvul,False,https://cdn.download.ams.birds.cornell.edu/api...,turvul
1,700658,NaN,NaN,Momotus coeruliceps,Blue-capped Motmot,NaN,José Alberto Lobato García,NaN,NaN,NaN,...,archive,https://ebird.org/profile/MjE0MTc3,448.0,286.0,https://macaulaylibrary.org/asset/700658,NaN,https://ebird.org/species/bucmot1,False,https://cdn.download.ams.birds.cornell.edu/api...,bucmot1
2,700666,NaN,NaN,Aptenodytes patagonicus,King Penguin,NaN,Laurent Demongin,NaN,NaN,NaN,...,archive,https://ebird.org/profile/NTAxNTM3,2176.0,3264.0,https://macaulaylibrary.org/asset/700666,NaN,https://ebird.org/species/kinpen1,False,https://cdn.download.ams.birds.cornell.edu/api...,kinpen1
3,700913,NaN,NaN,Anas capensis,Cape Teal,NaN,Megan Perkins,NaN,NaN,NaN,...,archive,NaN,575.0,850.0,https://macaulaylibrary.org/asset/700913,NaN,https://ebird.org/species/captea1,False,https://cdn.download.ams.birds.cornell.edu/api...,captea1
4,700917,NaN,NaN,Palmeria dolei,Akohekohe,NaN,Valerie Gebert,NaN,NaN,NaN,...,archive,https://ebird.org/profile/MTA1NDgy,1111.0,1631.0,https://macaulaylibrary.org/asset/700917,NaN,https://ebird.org/species/crehon,False,https://cdn.download.ams.birds.cornell.edu/api...,crehon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019957,264493941,NaN,NaN,Setophaga pensylvanica,Chestnut-sided Warbler,NaN,Teresa Noel,9/18/2020,2020.0,9.0,...,NaN,https://ebird.org/profile/MzA3MjEw,803.0,1072.0,https://macaulaylibrary.org/asset/264493941,https://ebird.org/view/checklist/S73736010,https://ebird.org/species/chswar,False,https://cdn.download.ams.birds.cornell.edu/api...,chswar
1019958,264493951,NaN,NaN,Setophaga pensylvanica,Chestnut-sided Warbler,NaN,Teresa Noel,9/18/2020,2020.0,9.0,...,NaN,https://ebird.org/profile/MzA3MjEw,736.0,981.0,https://macaulaylibrary.org/asset/264493951,https://ebird.org/view/checklist/S73736010,https://ebird.org/species/chswar,False,https://cdn.download.ams.birds.cornell.edu/api...,chswar
1019959,264493961,NaN,NaN,Setophaga pensylvanica,Chestnut-sided Warbler,NaN,Teresa Noel,9/18/2020,2020.0,9.0,...,NaN,https://ebird.org/profile/MzA3MjEw,783.0,1044.0,https://macaulaylibrary.org/asset/264493961,https://ebird.org/view/checklist/S73736010,https://ebird.org/species/chswar,False,https://cdn.download.ams.birds.cornell.edu/api...,chswar
1019960,264493971,NaN,NaN,Setophaga pensylvanica,Chestnut-sided Warbler,NaN,Teresa Noel,9/18/2020,2020.0,9.0,...,NaN,https://ebird.org/profile/MzA3MjEw,780.0,1040.0,https://macaulaylibrary.org/asset/264493971,https://ebird.org/view/checklist/S73736010,https://ebird.org/species/chswar,False,https://cdn.download.ams.birds.cornell.edu/api...,chswar


In [218]:

df_jpg100_meta = df_jpg100.merge(df_meta_, on=['ML Catalog Number'], how='left')
df_jpg100_meta.to_csv('meta/jpg_top100_meta3.csv')

- fiil value of 'Format' column

In [ ]:
# df_meta_format = pd.read_csv('meta/meta_all_mlcn_dropdup.csv', 
#                              usecols=['ML Catalog Number','Format'], 
#                              dtype={'ML Catalog Number': np.uint32, 'Format': 'category'},
#                              )

# df_Format = pd.merge(df_jpg100_meta['ML Catalog Number'],  df_meta_format,
#          on=['ML Catalog Number'], how='left')
# df_jpg100_meta['Format'] = df_Format['Format']
# df_jpg100_meta.to_csv('meta/jpg_top100_meta3.csv')

,ML Catalog Number,Format
0,114289801,Photo
1,114289821,Photo
2,114404941,Photo
3,114404951,Photo
4,115495451,Photo
...,...,...
896550,59630711,Photo
896551,59630721,Photo
896552,63554951,Photo
896553,79738531,Photo


- dtype conversion to reduce memory use
    - origin
        - dtypes: float64(24), object(37), uint32(1)
        - memory usage: 427.5+ MB
    - after conversion
        - dtypes: category(10), float16(13), float64(11), object(27), uint32(1)
        - memory usage: 301.0+ MB

In [220]:
df_jpg100_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896555 entries, 0 to 896554
Data columns (total 62 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ML Catalog Number            896555 non-null  uint32 
 1   Dir_FName                    896555 non-null  object 
 2   Parent_Dir                   896555 non-null  object 
 3   Size                         896555 non-null  float64
 4   Sci_N                        896555 non-null  object 
 5   Macaulay_public              37 non-null      float64
 6   Format                       3 non-null       object 
 7   Scientific Name              896535 non-null  object 
 8   Common Name                  896521 non-null  object 
 9   Background Species           1807 non-null    object 
 10  Recordist                    896478 non-null  object 
 11  Date                         884239 non-null  object 
 12  Year                         884239 non-null  float64
 13 

In [295]:
df_jpg100_meta.memory_usage(deep=True) / 1024 ** 2

Index                   6.840172
ML Catalog Number       3.420086
Dir_FName              84.817535
Parent_Dir             57.814425
Size                    3.420086
                         ...    
eBird Checklist URL    83.842659
ebird Species URL      74.992299
isInternalUser         27.360687
mediaDownloadUrl       96.614800
report_as              54.540119
Length: 63, dtype: float64

In [222]:
df_jpg100_meta.describe()

,ML Catalog Number,Size,Macaulay_public,Year,Month,Day,Time,Latitude,Longitude,Elevation (m),...,Water Temp(ºC),Class,Order,Taxonomic Sort,Recordist 2 Display Name,Recordist 2 Profile URL,Average Community Rating,Number of Ratings,Original Image Height,Original Image Width
count,8.965550e+05,896555.000000,37.000000,884239.000000,884239.000000,884239.000000,884090.000000,886648.000000,886648.000000,0.0,...,0.0,0.0,0.0,896521.00000,0.0,0.0,896517.000000,896521.000000,879330.000000,879330.000000
mean,1.572506e+08,195.597328,0.432432,2017.026498,6.354247,15.534699,834.940365,6.147125,-1.407771,NaN,...,NaN,NaN,NaN,16261.29193,NaN,NaN,3.766184,2.312475,1876.284360,2570.403069
std,6.648670e+07,172.323658,0.502247,3.480732,3.546743,8.786382,490.815387,25.165974,88.667509,NaN,...,NaN,NaN,NaN,10397.20468,NaN,NaN,1.362232,3.151649,992.005828,1398.922494
min,7.006500e+05,1.220000,0.000000,1810.000000,1.000000,1.000000,0.000000,-77.845100,-179.996100,NaN,...,NaN,NaN,NaN,1.00000,NaN,NaN,0.000000,0.000000,62.000000,75.000000
25%,1.063928e+08,130.190000,0.000000,2016.000000,3.000000,8.000000,615.000000,-12.870200,-77.096400,NaN,...,NaN,NaN,NaN,6903.00000,NaN,NaN,3.000000,1.000000,1055.000000,1406.000000
50%,1.704810e+08,177.150000,0.000000,2018.000000,6.000000,15.000000,800.000000,7.096000,-25.253900,NaN,...,NaN,NaN,NaN,15159.00000,NaN,NaN,4.000000,2.000000,1701.000000,2283.000000
75%,2.061341e+08,231.560000,1.000000,2019.000000,10.000000,23.000000,1150.000000,25.280400,79.529400,NaN,...,NaN,NaN,NaN,25976.00000,NaN,NaN,5.000000,3.000000,2592.000000,3611.000000
max,2.644940e+08,9048.920000,1.000000,2020.000000,12.000000,31.000000,2359.000000,82.499400,179.996100,NaN,...,NaN,NaN,NaN,34498.00000,NaN,NaN,5.000000,657.000000,16000.000000,18000.000000


In [19]:
# convert  dtype from obj to category
col_obj = df_jpg100_meta.columns[df_jpg100_meta.dtypes == 'object'].values
df_nunique = df_jpg100_meta[col_obj ].apply(lambda x: x.nunique())
col_obj = df_nunique[df_nunique<30].index.values
print(col_obj)
df_nunique[df_nunique<30]

col_float16 =  ['Macaulay_public', 'Year','Month','Day', 'Elevation (m)','Water Temp(ºC)','Air Temp(ºC)','Collected','Specimen ID', 
                'Average Community Rating', 'Number of Ratings','Original Image Height','Original Image Width']

column_dtypes = dict(
                    **{'ML Catalog Number': np.uint32},
                    **dict(zip(col_float16, [np.float16]*len(col_float16))),
                    **dict(zip(col_obj, ['category']*len(col_obj)))
                    )
column_dtypes

['Format' 'Age/Sex' 'Playback' 'Recorder' 'Microphone' 'Accessory'
 'Taxon Category' 'Asset State' 'isInternalUser' 'mediaDownloadUrl']


{'ML Catalog Number': numpy.uint32,
 'Macaulay_public': numpy.float16,
 'Year': numpy.float16,
 'Month': numpy.float16,
 'Day': numpy.float16,
 'Elevation (m)': numpy.float16,
 'Water Temp(ºC)': numpy.float16,
 'Air Temp(ºC)': numpy.float16,
 'Collected': numpy.float16,
 'Specimen ID': numpy.float16,
 'Average Community Rating': numpy.float16,
 'Number of Ratings': numpy.float16,
 'Original Image Height': numpy.float16,
 'Original Image Width': numpy.float16,
 'Format': 'category',
 'Age/Sex': 'category',
 'Playback': 'category',
 'Recorder': 'category',
 'Microphone': 'category',
 'Accessory': 'category',
 'Taxon Category': 'category',
 'Asset State': 'category',
 'isInternalUser': 'category',
 'mediaDownloadUrl': 'category'}

In [6]:
# df_jpg100_meta.to_csv('meta/jpg_top100_meta3.csv')
df_jpg100_meta = pd.read_csv('meta/jpg_top100_meta3.csv', index_col=0, 
                             dtype=column_dtypes
                             )
df_jpg100_meta.info()

<class 'modin.pandas.dataframe.DataFrame'>
Int64Index: 896555 entries, 0 to 896554
Data columns (total 62 columns):
 #   Column                       Non-Null Count   Dtype   
---  ---------------------------  ---------------  -----   
 0   ML Catalog Number            896555 non-null  uint32
 1   Dir_FName                    896555 non-null  object
 2   Parent_Dir                   896555 non-null  object
 3   Size                         896555 non-null  float64
 4   Sci_N                        896555 non-null  object
 5   Macaulay_public              37 non-null      float16
 6   Format                       3 non-null       category
 7   Scientific Name              896535 non-null  object
 8   Common Name                  896521 non-null  object
 9   Background Species           1807 non-null    object
 10  Recordist                    896478 non-null  object
 11  Date                         884239 non-null  object
 12  Year                         884239 non-null  float16
 13  

# Establish 'Label"

## Load Meta data
- meta file : 'jpg_top100_meta4.csv'

In [412]:
%%time
meta_file = dir_meta/'jpg_top100_meta4.csv'
df_jpg100_meta = pd.read_csv(meta_file, index_col=0,) 
                            #  dtype=column_dtypes
                             
col_basic = ['ML_Catalog_Number', 'Dir_FName', 'Macaulay_public']
col_file = ['Dir_FName', 'Parent_Dir', 'Sci_N']
col_taxon = ['Order', 'Family', 'Genus', 'SCI_NAME_2021', 'Sci_Name', 'Parent_Species', 'Taxon_Category']
col_geo = ['Country', 'Latitude', 'Longitude']
col_time = ['Year', 'Month', 'Day']
col_rating = ['Average_Community_Rating','Number_of_Ratings']
col_state = ['Age/Sex', 'Behaviors']
col_rating = ['Number_of_Ratings', 'Average_Community_Rating',]
df_meta = df_jpg100_meta[col_basic + col_taxon + col_rating + col_state].copy()
df_meta 

<timed exec>:2: DtypeWarning: Columns (10,31,32,33,60) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 7.06 s, sys: 41 s, total: 48 s
Wall time: 48.1 s


,ML_Catalog_Number,Dir_FName,Macaulay_public,Order,Family,Genus,SCI_NAME_2021,Sci_Name,Parent_Species,Taxon_Category,Number_of_Ratings,Average_Community_Rating,Age/Sex,Behaviors
0,114289801,Acanthisittidae/Acanthisitta_chloris_0_114289801,NaN,Passeriformes,Acanthisittidae,Acanthisitta,Acanthisitta chloris,Acanthisitta chloris,Acanthisitta chloris,Species,2.0,4.000,Unknown,NaN
1,114289821,Acanthisittidae/Acanthisitta_chloris_0_114289821,NaN,Passeriformes,Acanthisittidae,Acanthisitta,Acanthisitta chloris,Acanthisitta chloris,Acanthisitta chloris,Species,2.0,4.000,Unknown,NaN
2,114404941,Acanthisittidae/Acanthisitta_chloris_0_114404941,NaN,Passeriformes,Acanthisittidae,Acanthisitta,Acanthisitta chloris,Acanthisitta chloris,Acanthisitta chloris,Species,2.0,4.500,Unknown,NaN
3,114404951,Acanthisittidae/Acanthisitta_chloris_0_114404951,NaN,Passeriformes,Acanthisittidae,Acanthisitta,Acanthisitta chloris,Acanthisitta chloris,Acanthisitta chloris,Species,2.0,4.000,Unknown,NaN
4,115495451,Acanthisittidae/Acanthisitta_chloris_0_115495451,NaN,Passeriformes,Acanthisittidae,Acanthisitta,Acanthisitta chloris,Acanthisitta chloris,Acanthisitta chloris,Species,21.0,4.906,Unknown,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896550,59630711,Zosteropidae/Zosterornis_whiteheadi_0_59630711,NaN,Passeriformes,Zosteropidae,Zosterornis,Zosterornis whiteheadi,Zosterornis whiteheadi,Zosterornis whiteheadi,Species,1.0,3.000,Unknown,NaN
896551,59630721,Zosteropidae/Zosterornis_whiteheadi_0_59630721,NaN,Passeriformes,Zosteropidae,Zosterornis,Zosterornis whiteheadi,Zosterornis whiteheadi,Zosterornis whiteheadi,Species,0.0,0.000,Unknown,NaN
896552,63554951,Zosteropidae/Zosterornis_whiteheadi_0_63554951,NaN,Passeriformes,Zosteropidae,Zosterornis,Zosterornis whiteheadi,Zosterornis whiteheadi,Zosterornis whiteheadi,Species,1.0,3.000,Unknown,NaN
896553,79738531,Zosteropidae/Zosterornis_whiteheadi_0_79738531,NaN,Passeriformes,Zosteropidae,Zosterornis,Zosterornis whiteheadi,Zosterornis whiteheadi,Zosterornis whiteheadi,Species,4.0,3.000,Unknown,NaN


## Problemed data preprocess

### Same "Parent_Species" name with different Genus 
- because ("SCI_NAME_2021" is nan)  and ("SCI_NAME_2021" != "Parent_Species")

In [413]:
temp = df_meta.groupby('Parent_Species').Genus.nunique() 
problemed_list = temp[temp>1].sort_index().index.values
print(problemed_list)
temp[temp>1].sort_index()

['Glaucidium brodiei' 'Meliphaga gracilis' 'Sylvia cantillans'
 'Trichoglossus flavoviridis' 'Turdinus crispifrons']


Parent_Species
Glaucidium brodiei            2
Meliphaga gracilis            2
Sylvia cantillans             2
Trichoglossus flavoviridis    2
Turdinus crispifrons          2
Name: Genus, dtype: int64

In [414]:
problemed_list = problemed_list 
temp = (df_meta.query("Parent_Species.isin(@problemed_list)") 
        .sort_values('Parent_Species')[col_taxon])
temp.drop_duplicates(['Genus', 'Parent_Species'],  )
 

,Order,Family,Genus,SCI_NAME_2021,Sci_Name,Parent_Species,Taxon_Category
687505,Strigiformes,Strigidae,Taenioptynx,Taenioptynx brodiei,Glaucidium brodiei,Glaucidium brodiei,Species
687475,Strigiformes,Strigidae,Glaucidium,NaN,Glaucidium brodiei brodiei/pardalotum,Glaucidium brodiei,Group
386639,Passeriformes,Meliphagidae,Microptilotis,Microptilotis gracilis,Meliphaga gracilis,Meliphaga gracilis,Species
386705,Passeriformes,Meliphagidae,Meliphaga,NaN,Meliphaga gracilis gracilis,Meliphaga gracilis,Group
712350,Passeriformes,Sylviidae,Curruca,Curruca iberiae,Sylvia cantillans inornata,Sylvia cantillans,Group
712430,Passeriformes,Sylviidae,Sylvia,NaN,Sylvia cantillans albistriata,Sylvia cantillans,Group
624545,Psittaciformes,Psittaculidae,Saudareos,Saudareos meyeri,Trichoglossus flavoviridis meyeri,Trichoglossus flavoviridis,Group
624543,Psittaciformes,Psittaculidae,Trichoglossus,NaN,Trichoglossus flavoviridis,Trichoglossus flavoviridis,Species
515670,Passeriformes,Pellorneidae,Gypsophila,Gypsophila calcicola,Turdinus crispifrons calcicola,Turdinus crispifrons,Group
515729,Passeriformes,Pellorneidae,Turdinus,NaN,Turdinus crispifrons crispifrons/annamensis,Turdinus crispifrons,Group


In [415]:
for sp_ in problemed_list:
    genus_ = df_meta.query('(Parent_Species == @sp_) & SCI_NAME_2021.notnull()').Genus.unique()[0]
    index_ = df_meta[df_meta.Parent_Species == sp_].index.values
    df_meta.loc[index_, 'Genus'] = genus_

In [416]:
problemed_list  
temp = (df_meta.query("Parent_Species.isin(@problemed_list)") 
        .sort_values('Parent_Species')[col_taxon])
temp.drop_duplicates(['Genus', 'Parent_Species'], )
# temp

,Order,Family,Genus,SCI_NAME_2021,Sci_Name,Parent_Species,Taxon_Category
687505,Strigiformes,Strigidae,Taenioptynx,Taenioptynx brodiei,Glaucidium brodiei,Glaucidium brodiei,Species
386639,Passeriformes,Meliphagidae,Microptilotis,Microptilotis gracilis,Meliphaga gracilis,Meliphaga gracilis,Species
712350,Passeriformes,Sylviidae,Curruca,Curruca iberiae,Sylvia cantillans inornata,Sylvia cantillans,Group
624545,Psittaciformes,Psittaculidae,Saudareos,Saudareos meyeri,Trichoglossus flavoviridis meyeri,Trichoglossus flavoviridis,Group
515670,Passeriformes,Pellorneidae,Gypsophila,Gypsophila calcicola,Turdinus crispifrons calcicola,Turdinus crispifrons,Group


In [417]:
temp = df_meta.groupby('Parent_Species').Genus.nunique() 
problemed_list = temp[temp>1].sort_index().index.values
print(problemed_list)
temp[temp>1].sort_index()

[]


Series([], Name: Genus, dtype: int64)

### Fill Parent_Species.isnull() and SCI_NAME_2021.notnull()
- fill `Parent_Species` with  `SCI_NAME_2021`

In [418]:
temp = df_meta.query('Parent_Species.isnull() and SCI_NAME_2021.notnull()')
index_ =temp .index.values
sp_ =  temp.SCI_NAME_2021.unique()
print(sp_)

assert len(sp_ ) == 1
df_meta.loc[index_, 'Parent_Species'] = sp_[0]
df_meta.query('Parent_Species.isnull() and SCI_NAME_2021.notnull()')

['Brachypteryx montana erythrogyna']


,ML_Catalog_Number,Dir_FName,Macaulay_public,Order,Family,Genus,SCI_NAME_2021,Sci_Name,Parent_Species,Taxon_Category,Number_of_Ratings,Average_Community_Rating,Age/Sex,Behaviors


## 建立物種清單

### 物種篩選
- 分類單元以`Parent Species`欄位為依據
    - 物種(屬名 種名)
- ~~物種選擇依據`Taxon Category`欄位~~
    - selected: 'Species', 'Group', 'Intergrade', 'Form', 'Domestic'
    - 'Domestic' :  取前兩個單字，以'_'相連。取代 'sp.' 為 'sp'
    - excluded: 'Spuh', 'Slash', 'Hybrid', 
    - 'Spuh', 'Slash' 設定類別為  Genus_sp

- 保留不明確的分類單元
    - 類別數量會增加、acc1表現相對可能會較差
    - 若模型學習到辨識細微差異時，預期可以分配至不同的高維空間，以讓後續全連接層在分類預測時做出區別
    
- `"Dir_class"`欄位建立：
    - 使用meta資料中`"Parent_Species"`欄位，非null的資料
        - 內含有 '/', ' x ', '()', 'sp.', '(Domestic type)'等
    - 納入`Taxon Category`中'Spuh', 'Slash', 'Hybrid'等類群無法確認的類別
    - 物種類別以`"Parent_Species"`為根據 (目、科、屬，其他欄位主要根據2021年資料關聯建立)
        - 包含~~排除~~有'/', ' x '的資料
        - 把括號後的文字移除 
        - 'sp.' 取代為 'sp'
        - 屬名無法確認(NaN)的資料排除
        - `"Parent_Species"`缺失的資料以"SCI_NAME_2021"取代
    
- `Label` 欄位建立
    - 按 `"Order"` +  `"Family"` +  `"Genus"` + `"Dir_class"排序建立
- 建立 `"Dir_class_full"` 
    - `"Dir_class_full"` = `"Label"`  + `"Order"` +  `"Family"` +  `"Genus"` + `"Dir_class"`

In [128]:
# df_meta.dropna(subset='Parent_Species').query("Parent_Species.str.contains('/')")

- filter  'Species', 'Group', 'Intergrade', 'Form', 'Domestic'

### Establish class name according `Parent_Species` column

#### `def sciName_process()`

In [419]:
def sciName_process(name:str):
    return name.split('(')[0].strip(' ').replace('sp.', 'sp').replace(' ', '_').replace('/', '_')
    # .replace('x', '_')

In [420]:
mask = df_meta.query('Parent_Species.notnull()').index
df_meta.loc[mask, 'Dir_class'] = (df_meta.loc[mask].Parent_Species.apply(sciName_process))
df_meta[['Parent_Species', 'Dir_class']]

,Parent_Species,Dir_class
0,Acanthisitta chloris,Acanthisitta_chloris
1,Acanthisitta chloris,Acanthisitta_chloris
2,Acanthisitta chloris,Acanthisitta_chloris
3,Acanthisitta chloris,Acanthisitta_chloris
4,Acanthisitta chloris,Acanthisitta_chloris
...,...,...
896550,Zosterornis whiteheadi,Zosterornis_whiteheadi
896551,Zosterornis whiteheadi,Zosterornis_whiteheadi
896552,Zosterornis whiteheadi,Zosterornis_whiteheadi
896553,Zosterornis whiteheadi,Zosterornis_whiteheadi


In [343]:
(df_meta.dropna(subset='Parent_Species')
 # .query('Parent_Species.str.contains("/") or Parent_Species.str.contains(" x ")')
  # .query("Parent_Species.str.contains('\(')")
 [['Parent_Species', 'Dir_class']]
 .Dir_class.nunique()
)

11193

In [528]:
col_ = ['Order', 'Family', 'Genus',  'Dir_class']
class_count = (df_meta.groupby(col_).ML_Catalog_Number.count().rename('Count')
               .reset_index().rename_axis("Label").reset_index())
print(class_count.Dir_class.nunique())
print(class_count.Label.nunique())
print(class_count.Label.dtype)
class_count 

11186
11186
int64


,Label,Order,Family,Genus,Dir_class,Count
0,0,Accipitriformes,Accipitridae,Accipiter,Accipiter_albogularis,14
1,1,Accipitriformes,Accipitridae,Accipiter,Accipiter_badius,233
2,2,Accipitriformes,Accipitridae,Accipiter,Accipiter_bicolor,189
3,3,Accipitriformes,Accipitridae,Accipiter,Accipiter_brevipes,99
4,4,Accipitriformes,Accipitridae,Accipiter,Accipiter_brevipes_nisus,5
...,...,...,...,...,...,...
11181,11181,Trogoniformes,Trogonidae,Trogon,Trogon_ramonianus,100
11182,11182,Trogoniformes,Trogonidae,Trogon,Trogon_rufus,119
11183,11183,Trogoniformes,Trogonidae,Trogon,Trogon_surrucura,273
11184,11184,Trogoniformes,Trogonidae,Trogon,Trogon_violaceus,100


In [529]:
path = 'meta/label_class_count.csv'
class_count.to_csv(path)
class_count = pd.read_csv(path, index_col=0)
class_count

,Label,Order,Family,Genus,Dir_class,Count
0,0,Accipitriformes,Accipitridae,Accipiter,Accipiter_albogularis,14
1,1,Accipitriformes,Accipitridae,Accipiter,Accipiter_badius,233
2,2,Accipitriformes,Accipitridae,Accipiter,Accipiter_bicolor,189
3,3,Accipitriformes,Accipitridae,Accipiter,Accipiter_brevipes,99
4,4,Accipitriformes,Accipitridae,Accipiter,Accipiter_brevipes_nisus,5
...,...,...,...,...,...,...
11181,11181,Trogoniformes,Trogonidae,Trogon,Trogon_ramonianus,100
11182,11182,Trogoniformes,Trogonidae,Trogon,Trogon_rufus,119
11183,11183,Trogoniformes,Trogonidae,Trogon,Trogon_surrucura,273
11184,11184,Trogoniformes,Trogonidae,Trogon,Trogon_violaceus,100


### Assign Genus uncornfirmed data  as np.nan
- ['Geranoaetus/Buteo sp.', 'Buteo/eagle sp.', nan, 'Anser/Branta sp.', 'Acanthis/Spinus sp.', 'Synoicus/Coturnix sp.', 'Gallinula/Fulica/Porphyrio sp.', 'Porzana/Zapornia/Hapalocrex sp.']

In [530]:
print(df_meta.loc[mask].Parent_Species.unique())
mask = df_meta.Genus.isnull() 
df_meta.loc[mask, 'Dir_class'] = np.nan

[]


In [531]:
(df_meta.dropna(subset='Parent_Species')
 # .query('Parent_Species.str.contains("/") or Parent_Species.str.contains(" x ")')
  # .query("Parent_Species.str.contains('\(')")
 [['Parent_Species', 'Dir_class']]
 .Dir_class.nunique()
)

11186

In [534]:
col_ = ['Order', 'Family', 'Genus',  'Dir_class']
class_count = (df_meta.groupby(col_).Dir_class.count().rename('Count')
               .reset_index().rename_axis("Label").reset_index())
class_count.Label = class_count.Label.apply(lambda x: f'{x:0>5d}')
class_count['Dir_class_full'] = (class_count.Label
                                 + '_'	+ class_count.Order + '_'+ class_count.Family + '_' + class_count.Genus + '_'	+ class_count.Dir_class)
class_count

,Label,Order,Family,Genus,Dir_class,Count,Dir_class_full
0,00000,Accipitriformes,Accipitridae,Accipiter,Accipiter_albogularis,14,00000_Accipitriformes_Accipitridae_Accipiter_A...
1,00001,Accipitriformes,Accipitridae,Accipiter,Accipiter_badius,233,00001_Accipitriformes_Accipitridae_Accipiter_A...
2,00002,Accipitriformes,Accipitridae,Accipiter,Accipiter_bicolor,189,00002_Accipitriformes_Accipitridae_Accipiter_A...
3,00003,Accipitriformes,Accipitridae,Accipiter,Accipiter_brevipes,99,00003_Accipitriformes_Accipitridae_Accipiter_A...
4,00004,Accipitriformes,Accipitridae,Accipiter,Accipiter_brevipes_nisus,5,00004_Accipitriformes_Accipitridae_Accipiter_A...
...,...,...,...,...,...,...,...
11181,11181,Trogoniformes,Trogonidae,Trogon,Trogon_ramonianus,100,11181_Trogoniformes_Trogonidae_Trogon_Trogon_r...
11182,11182,Trogoniformes,Trogonidae,Trogon,Trogon_rufus,119,11182_Trogoniformes_Trogonidae_Trogon_Trogon_r...
11183,11183,Trogoniformes,Trogonidae,Trogon,Trogon_surrucura,273,11183_Trogoniformes_Trogonidae_Trogon_Trogon_s...
11184,11184,Trogoniformes,Trogonidae,Trogon,Trogon_violaceus,100,11184_Trogoniformes_Trogonidae_Trogon_Trogon_v...


In [515]:
(class_count[class_count.Dir_class.duplicated(keep=False)]
# .merge(df_meta[['Dir_class', 'SCI_NAME_2021']], on='Dir_class')
#  .drop_duplicates(subset=['Dir_class', 'SCI_NAME_2021'])
)

,Label,Order,Family,Genus,Dir_class,Count,Dir_class_full


In [516]:
df_meta.Dir_class.nunique(), class_count.Dir_class.nunique()

(11186, 11186)

In [517]:
set(df_meta.Dir_class.unique())  - set(class_count.Dir_class.unique()) 

{nan}

### Establish df_label

In [522]:
df_label = pd.merge(class_count[['Label', 'Dir_class', 'Dir_class_full']], df_meta[['ML_Catalog_Number', 'Dir_class']], on= 'Dir_class', how='right')
print(df_label.Dir_class.nunique())
df_label

11186


,Label,Dir_class,Dir_class_full,ML_Catalog_Number
0,03090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114289801
1,03090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114289821
2,03090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114404941
3,03090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114404951
4,03090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,115495451
...,...,...,...,...
896550,09660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,59630711
896551,09660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,59630721
896552,09660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,63554951
896553,09660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,79738531


In [535]:
mask = df_label.Label.isnull()
df_label.loc[mask, 'Label'] = -1

df_label.Label = df_label.Label.astype(np.int64)

#### Check sp name with "/" and "_x_"

In [491]:
temp = df_meta.dropna(subset=['Parent_Species']).query('Parent_Species.str.contains("/")')
temp_sp_slash = temp.Parent_Species.unique()
print(f'Number of sp name("Parent_Species") contains "\\" : {temp.Parent_Species.nunique()}')
print(f'\tTaxon_Category: {temp.Taxon_Category.unique()}')
temp = df_meta.dropna(subset=['Parent_Species']).query('Parent_Species.str.contains(" x ")')   # 'Taxon_Category' is all belong 'hybrid'
temp_sp_hybrid = temp.Parent_Species.unique()
print(f'Number of sp name("Parent_Species") contains " x ": {temp.Parent_Species.nunique()}')
print(f'\tTaxon_Category: {temp.Taxon_Category.unique()}')

sp_list = set(temp_sp_slash) | set(temp_sp_hybrid)
print('Number of class("Parent_Species")  contains  "\\" or " x " :', len(sp_list))

set(temp_sp_slash) & set(temp_sp_hybrid)

Number of sp name("Parent_Species") contains "\" : 508
	Taxon_Category: ['Slash' 'Spuh' 'Hybrid' 'Form' 'Group']
Number of sp name("Parent_Species") contains " x ": 334
	Taxon_Category: ['Hybrid']
Number of class("Parent_Species")  contains  "\" or " x " : 835


{'Anas platyrhynchos x diazi/fulvigula',
 'Anser caerulescens/rossii x Branta hutchinsii/canadensis',
 'Aythya americana x marila/affinis',
 'Aythya collaris x marila/affinis',
 'Aythya fuligula x marila/affinis',
 'Bucephala albeola x clangula/islandica',
 'Bucephala clangula/islandica x Lophodytes cucullatus'}

In [505]:
print("Taxon_Category in ['Species', 'Group', 'Intergrade', 'Form', 'Domestic'], but  sp name('Parent_Species') contains '\\' : ")
mask = df_meta.Taxon_Category.isin(['Species', 'Group', 'Intergrade', 'Form', 'Domestic'])
df_meta[mask].query('Parent_Species.str.contains("\/")').drop_duplicates(['Parent_Species', 'Taxon_Category'])[['SCI_NAME_2021', 'Parent_Species', 'Taxon_Category']]

Taxon_Category in ['Species', 'Group', 'Intergrade', 'Form', 'Domestic'], but  sp name('Parent_Species') contains '\' : 


,SCI_NAME_2021,Parent_Species,Taxon_Category
319562,Hydrobates leucorhous/socorroensis (dark-rumped),Oceanodroma leucorhoa/socorroensis (dark-rumped),Form
319569,Hydrobates leucorhous/socorroensis (white-rumped),Oceanodroma leucorhoa/socorroensis (white-rumped),Form
566955,Picus viridis viridis/karelini,Picus viridis/sharpei,Group
590239,Ramphocaenus melanurus [melanurus Group],Ramphocaenus melanurus/sticturus,Group
597536,Puffinus assimilis haurakiensis,Puffinus assimilis/elegans,Group
597586,Puffinus bailloni bailloni,Puffinus bannermani/bailloni,Group
712861,Curruca curruca curruca/blythi,Sylvia curruca curruca/blythi/halimodendri,Group


In [478]:
mask = df_meta.Parent_Species.isin(temp_sp_slash)
df_meta[mask].groupby(['Family', 'Genus', 'Parent_Species']).ML_Catalog_Number.count().sort_values()

Family             Genus       Parent_Species                  
Laridae            Rissa       Rissa tridactyla/brevirostris         1
Furnariidae        Xenops      Xenops tenuirostris/rutilans          1
Threskiornithidae  Plegadis    Plegadis chihi/ridgwayi               1
Furnariidae        Synallaxis  Synallaxis azarae/courseni            1
Thraupidae         Poospiza    Poospiza whitii/nigrorufa             1
                                                                  ... 
Icteridae          Icterus     Icterus bullockii/galbula           100
Psittacidae        Psittacara  Psittacara mitratus/erythrogenys    100
Ardeidae           Ardea       Ardea alba/intermedia               100
Sulidae            Sula        Sula dactylatra/granti              100
Apodidae           Chaetura    Chaetura pelagica/vauxi             100
Name: ML_Catalog_Number, Length: 501, dtype: int64

#### merge label and meta

In [536]:
meta_file = dir_meta/'jpg_top100_meta4.csv'
df_jpg100_meta = pd.read_csv(meta_file, index_col=0,) 
df_jpg100_meta 

/tmp/ipykernel_83040/513086560.py:2: DtypeWarning: Columns (10,31,32,33,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df_jpg100_meta = pd.read_csv(meta_file, index_col=0,)


,ML_Catalog_Number,Dir_FName,Parent_Dir,Size,Sci_N,Macaulay_public,Format,Sci_Name,Common_Name,Background_Species,...,Original_Image_Height,Original_Image_Width,Specimen_Page_URL,eBird_Checklist_URL,ebird_Species_URL,isInternalUser,mediaDownloadUrl,report_as,Genus,SCI_NAME_2021
0,114289801,Acanthisittidae/Acanthisitta_chloris_0_114289801,Acanthisittidae,287.80,Acanthisitta_chloris_0,NaN,Photo,Acanthisitta chloris,Rifleman,NaN,...,2836.0,4120.0,https://macaulaylibrary.org/asset/114289801,https://ebird.org/view/checklist/S26591349,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
1,114289821,Acanthisittidae/Acanthisitta_chloris_0_114289821,Acanthisittidae,205.43,Acanthisitta_chloris_0,NaN,Photo,Acanthisitta chloris,Rifleman,NaN,...,4000.0,6000.0,https://macaulaylibrary.org/asset/114289821,https://ebird.org/view/checklist/S26591349,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
2,114404941,Acanthisittidae/Acanthisitta_chloris_0_114404941,Acanthisittidae,335.58,Acanthisitta_chloris_0,NaN,Photo,Acanthisitta chloris,Rifleman,NaN,...,1665.0,2500.0,https://macaulaylibrary.org/asset/114404941,https://ebird.org/view/checklist/S48312775,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
3,114404951,Acanthisittidae/Acanthisitta_chloris_0_114404951,Acanthisittidae,261.37,Acanthisitta_chloris_0,NaN,Photo,Acanthisitta chloris,Rifleman,NaN,...,1669.0,2500.0,https://macaulaylibrary.org/asset/114404951,https://ebird.org/view/checklist/S48312775,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
4,115495451,Acanthisittidae/Acanthisitta_chloris_0_115495451,Acanthisittidae,97.31,Acanthisitta_chloris_0,NaN,Photo,Acanthisitta chloris,Rifleman,NaN,...,1565.0,2400.0,https://macaulaylibrary.org/asset/115495451,https://ebird.org/view/checklist/S48500276,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896550,59630711,Zosteropidae/Zosterornis_whiteheadi_0_59630711,Zosteropidae,211.91,Zosterornis_whiteheadi_0,NaN,Photo,Zosterornis whiteheadi,Chestnut-faced Babbler,NaN,...,1983.0,2976.0,https://macaulaylibrary.org/asset/59630711,https://ebird.org/view/checklist/S35659495,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi
896551,59630721,Zosteropidae/Zosterornis_whiteheadi_0_59630721,Zosteropidae,194.77,Zosterornis_whiteheadi_0,NaN,Photo,Zosterornis whiteheadi,Chestnut-faced Babbler,NaN,...,1848.0,2772.0,https://macaulaylibrary.org/asset/59630721,https://ebird.org/view/checklist/S35659495,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi
896552,63554951,Zosteropidae/Zosterornis_whiteheadi_0_63554951,Zosteropidae,254.81,Zosterornis_whiteheadi_0,NaN,Photo,Zosterornis whiteheadi,Chestnut-faced Babbler,NaN,...,841.0,1500.0,https://macaulaylibrary.org/asset/63554951,https://ebird.org/view/checklist/S38159117,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi
896553,79738531,Zosteropidae/Zosterornis_whiteheadi_0_79738531,Zosteropidae,251.04,Zosterornis_whiteheadi_0,NaN,Photo,Zosterornis whiteheadi,Chestnut-faced Babbler,NaN,...,829.0,1500.0,https://macaulaylibrary.org/asset/79738531,https://ebird.org/view/checklist/S41419807,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi


## Establish df_meta_v5  

In [537]:
df_meta_v5 = pd.merge(df_label, df_jpg100_meta , on='ML_Catalog_Number')
df_meta_v5

,Label,Dir_class,Dir_class_full,ML_Catalog_Number,Dir_FName,Parent_Dir,Size,Sci_N,Macaulay_public,Format,...,Original_Image_Height,Original_Image_Width,Specimen_Page_URL,eBird_Checklist_URL,ebird_Species_URL,isInternalUser,mediaDownloadUrl,report_as,Genus,SCI_NAME_2021
0,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114289801,Acanthisittidae/Acanthisitta_chloris_0_114289801,Acanthisittidae,287.80,Acanthisitta_chloris_0,NaN,Photo,...,2836.0,4120.0,https://macaulaylibrary.org/asset/114289801,https://ebird.org/view/checklist/S26591349,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
1,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114289821,Acanthisittidae/Acanthisitta_chloris_0_114289821,Acanthisittidae,205.43,Acanthisitta_chloris_0,NaN,Photo,...,4000.0,6000.0,https://macaulaylibrary.org/asset/114289821,https://ebird.org/view/checklist/S26591349,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
2,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114404941,Acanthisittidae/Acanthisitta_chloris_0_114404941,Acanthisittidae,335.58,Acanthisitta_chloris_0,NaN,Photo,...,1665.0,2500.0,https://macaulaylibrary.org/asset/114404941,https://ebird.org/view/checklist/S48312775,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
3,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114404951,Acanthisittidae/Acanthisitta_chloris_0_114404951,Acanthisittidae,261.37,Acanthisitta_chloris_0,NaN,Photo,...,1669.0,2500.0,https://macaulaylibrary.org/asset/114404951,https://ebird.org/view/checklist/S48312775,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
4,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,115495451,Acanthisittidae/Acanthisitta_chloris_0_115495451,Acanthisittidae,97.31,Acanthisitta_chloris_0,NaN,Photo,...,1565.0,2400.0,https://macaulaylibrary.org/asset/115495451,https://ebird.org/view/checklist/S48500276,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896550,9660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,59630711,Zosteropidae/Zosterornis_whiteheadi_0_59630711,Zosteropidae,211.91,Zosterornis_whiteheadi_0,NaN,Photo,...,1983.0,2976.0,https://macaulaylibrary.org/asset/59630711,https://ebird.org/view/checklist/S35659495,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi
896551,9660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,59630721,Zosteropidae/Zosterornis_whiteheadi_0_59630721,Zosteropidae,194.77,Zosterornis_whiteheadi_0,NaN,Photo,...,1848.0,2772.0,https://macaulaylibrary.org/asset/59630721,https://ebird.org/view/checklist/S35659495,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi
896552,9660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,63554951,Zosteropidae/Zosterornis_whiteheadi_0_63554951,Zosteropidae,254.81,Zosterornis_whiteheadi_0,NaN,Photo,...,841.0,1500.0,https://macaulaylibrary.org/asset/63554951,https://ebird.org/view/checklist/S38159117,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi
896553,9660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,79738531,Zosteropidae/Zosterornis_whiteheadi_0_79738531,Zosteropidae,251.04,Zosterornis_whiteheadi_0,NaN,Photo

In [538]:
print(df_meta_v5.Label.dtype)

int64


In [539]:
meta_file = dir_meta/'jpg_top100_meta5.csv'
df_meta_v5.to_csv(meta_file)
df_meta_v5 = pd.read_csv(meta_file, index_col=0)
df_meta_v5.info()

/tmp/ipykernel_83040/4221068166.py:3: DtypeWarning: Columns (13,34,35,36,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta_v5 = pd.read_csv(meta_file, index_col=0)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 896555 entries, 0 to 896554
Data columns (total 67 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Label                        896555 non-null  int64  
 1   Dir_class                    896195 non-null  object 
 2   Dir_class_full               896195 non-null  object 
 3   ML_Catalog_Number            896555 non-null  int64  
 4   Dir_FName                    896555 non-null  object 
 5   Parent_Dir                   896555 non-null  object 
 6   Size                         896555 non-null  float64
 7   Sci_N                        896555 non-null  object 
 8   Macaulay_public              37 non-null      float64
 9   Format                       896521 non-null  object 
 10  Sci_Name                     896535 non-null  object 
 11  Common_Name                  896521 non-null  object 
 12  Background_Species           1807 non-null    object 
 13 

- Size of Label null : 373
    - Parent_Species.notnull() : 326
        - Genus could not be confirmed
    - Parent_Species.notnull() : 34
        - Genus could not be confirmed : 20
        - SCI_NAME_2021  could not be confirmed : 34

In [494]:
print(f'Size of Label null : ' , len(df_meta_v5.query('Dir_class.isnull()')))
print(f'\t Parent_Species.notnull() : ' , len(df_meta_v5.query('Dir_class.isnull() and Parent_Species.notnull()')))
print(f'\t\t- Genus data uncorfirmed')
print(f'\t Parent_Species.isnull(): ' , len(df_meta_v5.query('Dir_class.isnull() and Parent_Species.isnull()')))
print(f'\t\t SCI_NAME_2021.isnull(): ' , len(df_meta_v5.query('Dir_class.isnull() and Parent_Species.isnull() and SCI_NAME_2021.isnull()')))
print(f'\t\t Genus.isnull(): ' , len(df_meta_v5.query('Dir_class.isnull() and Parent_Species.isnull() and Genus.isnull()')))
# df_meta_v5.query('Dir_class.isnull() and  Parent_Species.isnull() and Genus.notnull() and SCI_NAME_2021.notnull()')[col_taxon] 

Size of Label null :  360
	 Parent_Species.notnull() :  326
		- Genus data uncorfirmed
	 Parent_Species.isnull():  34
		 SCI_NAME_2021.isnull():  34
		 Genus.isnull():  20


In [3]:
%%time
meta_file = dir_meta/'jpg_top100_meta5.csv'
df_meta_full = pd.read_csv(meta_file, index_col=0)

<timed exec>:2: DtypeWarning: Columns (13,34,35,36,63) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 7.48 s, sys: 767 ms, total: 8.25 s
Wall time: 8.26 s


In [5]:
%%time
meta_file = dir_meta/'jpg_top100_meta5.feather'
df_meta_full.to_feather(meta_file)

CPU times: user 2.86 s, sys: 730 ms, total: 3.59 s
Wall time: 2 s


In [7]:
%%time
df_meta_full = pd.read_feather(meta_file)
df_meta_full

CPU times: user 1.97 s, sys: 2.17 s, total: 4.14 s
Wall time: 1.6 s


,Label,Dir_class,Dir_class_full,ML_Catalog_Number,Dir_FName,Parent_Dir,Size,Sci_N,Macaulay_public,Format,...,Original_Image_Height,Original_Image_Width,Specimen_Page_URL,eBird_Checklist_URL,ebird_Species_URL,isInternalUser,mediaDownloadUrl,report_as,Genus,SCI_NAME_2021
0,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114289801,Acanthisittidae/Acanthisitta_chloris_0_114289801,Acanthisittidae,287.80,Acanthisitta_chloris_0,NaN,Photo,...,2836.0,4120.0,https://macaulaylibrary.org/asset/114289801,https://ebird.org/view/checklist/S26591349,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
1,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114289821,Acanthisittidae/Acanthisitta_chloris_0_114289821,Acanthisittidae,205.43,Acanthisitta_chloris_0,NaN,Photo,...,4000.0,6000.0,https://macaulaylibrary.org/asset/114289821,https://ebird.org/view/checklist/S26591349,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
2,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114404941,Acanthisittidae/Acanthisitta_chloris_0_114404941,Acanthisittidae,335.58,Acanthisitta_chloris_0,NaN,Photo,...,1665.0,2500.0,https://macaulaylibrary.org/asset/114404941,https://ebird.org/view/checklist/S48312775,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
3,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,114404951,Acanthisittidae/Acanthisitta_chloris_0_114404951,Acanthisittidae,261.37,Acanthisitta_chloris_0,NaN,Photo,...,1669.0,2500.0,https://macaulaylibrary.org/asset/114404951,https://ebird.org/view/checklist/S48312775,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
4,3090,Acanthisitta_chloris,03090_Passeriformes_Acanthisittidae_Acanthisit...,115495451,Acanthisittidae/Acanthisitta_chloris_0_115495451,Acanthisittidae,97.31,Acanthisitta_chloris_0,NaN,Photo,...,1565.0,2400.0,https://macaulaylibrary.org/asset/115495451,https://ebird.org/view/checklist/S48500276,https://ebird.org/species/riflem1,False,https://cdn.download.ams.birds.cornell.edu/api...,riflem1,Acanthisitta,Acanthisitta chloris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896550,9660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,59630711,Zosteropidae/Zosterornis_whiteheadi_0_59630711,Zosteropidae,211.91,Zosterornis_whiteheadi_0,NaN,Photo,...,1983.0,2976.0,https://macaulaylibrary.org/asset/59630711,https://ebird.org/view/checklist/S35659495,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi
896551,9660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,59630721,Zosteropidae/Zosterornis_whiteheadi_0_59630721,Zosteropidae,194.77,Zosterornis_whiteheadi_0,NaN,Photo,...,1848.0,2772.0,https://macaulaylibrary.org/asset/59630721,https://ebird.org/view/checklist/S35659495,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi
896552,9660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,63554951,Zosteropidae/Zosterornis_whiteheadi_0_63554951,Zosteropidae,254.81,Zosterornis_whiteheadi_0,NaN,Photo,...,841.0,1500.0,https://macaulaylibrary.org/asset/63554951,https://ebird.org/view/checklist/S38159117,https://ebird.org/species/chfbab1,False,https://cdn.download.ams.birds.cornell.edu/api...,chfbab1,Zosterornis,Zosterornis whiteheadi
896553,9660,Zosterornis_whiteheadi,09660_Passeriformes_Zosteropidae_Zosterornis_Z...,79738531,Zosteropidae/Zosterornis_whiteheadi_0_79738531,Zosteropidae,251.04,Zosterornis_whiteheadi_0,NaN,Photo

---

# Expolre meta

In [8]:
meta_file = dir_meta/'jpg_top100_meta5.feather'
df_meta_full = pd.read_feather(meta_file)

In [6]:
df_meta_full.columns

Index(['Label', 'Dir_class', 'Dir_class_full', 'ML_Catalog_Number',
       'Dir_FName', 'Parent_Dir', 'Size', 'Sci_N', 'Macaulay_public', 'Format',
       'Sci_Name', 'Common_Name', 'Background_Species', 'Recordist', 'Date',
       'Year', 'Month', 'Day', 'Time', 'Country', 'Country-State-County',
       'State', 'County', 'Locality', 'Latitude', 'Longitude', 'Elevation_(m)',
       'Age/Sex', 'Behaviors', 'Playback', 'Collected', 'Specimen_ID',
       'Home_Archive_Catalog_Number', 'Recorder', 'Microphone', 'Accessory',
       'Partner_Institution', 'eBird_Checklist_ID', 'Unconfirmed?',
       'Air_Temp(ºC)', 'Water_Temp(ºC)', 'Media_notes', 'Observation_Details',
       'Class', 'Order', 'Family', 'Parent_Species', 'eBird_Species_Code',
       'Taxon_Category', 'Taxonomic_Sort', 'Recordist_2_Display_Name',
       'Recordist_2_Profile_URL', 'Average_Community_Rating',
       'Number_of_Ratings', 'Asset_State', 'Asset_Tags',
       'Contributor_Profile_URL', 'Original_Image_Height',
  

### Sample original imgs

In [ ]:
taxon = 'Order'
# cls_ = 'Rhagologidae'
out_size = 128
n_smp = 512

cls_list = df_meta[taxon].dropna().sort_values().unique()

for i, cls_ in enumerate(cls_list):
    idx = df_meta.query(f'{taxon} == @cls_').index.values

    files_path_  = list(data_path + '/' +  val_data + '/' + df_meta.loc[idx].Dir_FName	+ '.jpg')
    labels = df_meta.loc[idx].Label.values

    assert len(files_path_) == len(labels) 
    
    batch_size = len(files_path_) if len(files_path_) < n_smp else n_smp

    # simple augmentation

    transform_toTensor = transforms.Compose([transforms.Resize(out_size, interpolation=PIL.Image.BICUBIC), 
                                             transforms.CenterCrop(out_size),
                                             transforms.ToTensor()])
    dataset = util_datasets.ImageDatasetFromFile(files_path_, labels, transform=transform_toTensor)
    data_loader = torch.utils.data.DataLoader(dataset, drop_last=True, shuffle=True, batch_size=batch_size)
    imgs_, labels_ = next(iter(data_loader))

    save_path = Path(f'temp/sample_imgs/{taxon}')
    if not save_path.exists():
        save_path.mkdir(parents=True, )
    nrow = 16
    vutils.save_image(imgs_, save_path/f'{taxon}_{cls_}{batch_size}.jpg', nrow=nrow)
    
    print(f'{i:,d}, Data size : {len(files_path_ )}, {taxon}, {cls_:30s}', end='\r')

## `Asset_Tags`

In [16]:
print('Asset_Tags.notnull() : ',  df_meta_full.query('Asset_Tags.notnull()').index.size)
asset_Tags_list = df_meta_full.Asset_Tags.dropna().unique()
print('Number of asset_Tags : ', len(asset_Tags_list))

Asset_Tags.notnull() :  26053
Number of asset_Tags :  32


In [14]:
df_meta_full.groupby('Asset_Tags').ML_Catalog_Number.count()

Asset_Tags
archive                                                                                2750
archive; watermark                                                                        3
back_of_camera                                                                          111
back_of_camera; field_notes_sketch; habitat; multiple_species; non_bird; watermark        2
back_of_camera; habitat                                                                   2
back_of_camera; habitat; multiple_species; watermark                                      1
back_of_camera; multiple_species                                                          1
back_of_camera; non_bird                                                                  4
back_of_camera; watermark                                                                 7
field_notes_sketch                                                                      238
field_notes_sketch; habitat                                          

In [18]:
df_meta_full.dropna(subset=['Asset_Tags']).query('Asset_Tags.str.contains("field_notes_sketch")').groupby(['Family']).size().sort_values()

Family
Momotidae          1
Eurylaimidae       1
Stercorariidae     1
Stenostiridae      1
Spheniscidae       1
                  ..
Turdidae          12
Psittacidae       13
Picidae           13
Fringillidae      14
Columbidae        18
Length: 119, dtype: int64

#### 'field_notes_sketch'
- Some of the image data marked with 'field_notes_sketch' are sketch images  

<img src="temp/Asset_Tags/field_notes_sketch_24.jpg" width="600"/>

In [57]:
import PIL
import torch
# from torch import nn
import torchvision.transforms as transforms
# import torchvision.datasets as datasets
import torchvision.utils as vutils
import torch.utils.data as data

class ImageDatasetFromFile(data.Dataset):
    def __init__(self, image_path:list, 
                input_height=256, input_width=256, output_height=128, output_width=128, transform=None):
        super(ImageDatasetFromFile, self).__init__()
        self.image_path = image_path
        # self.labels = labels
        self.transform = transform
        self.toTensor = transforms.Compose([transforms.ToTensor()])

    def __getitem__(self, index):
        img_ = PIL.Image.open(self.image_path[index]).convert('RGB')
        img = self.toTensor(img_) if self.transform is None else self.transform(img_) 
        # label = self.labels[index]
        return img 

    def __len__(self):
        return len(self.image_path)
transform_toTensor = transforms.Compose([transforms.Resize(128, interpolation=PIL.Image.BICUBIC), 
                                         transforms.CenterCrop(128),
                                         transforms.ToTensor()])

In [74]:
root =  'downloaded/jpg_top100'
n_smp = 24

for tag_ in asset_Tags_list:
    print(tag_, ' : ', len(idx_))
    mask = df_meta_full.Asset_Tags == tag_
    idx_ = df_meta_full[mask].index.values
    
    mlcns = df_meta_full[mask].ML_Catalog_Number.values
    files_path_  = list(root + '/' + df_meta_full[mask].Dir_FName + '.jpg')
    print(len(files_path_))
    if len(files_path_) > n_smp:
        files_path_ = np.random.choice(files_path_, size=n_sample)
    
    batch_size = len(files_path_) if len(files_path_) < n_smp else n_smp

    # simple augmentation

    dataset = ImageDatasetFromFile(files_path_, transform=transform_toTensor)
    data_loader = torch.utils.data.DataLoader(dataset, drop_last=True, shuffle=True, batch_size=batch_size)
    imgs_ = next(iter(data_loader))

    save_path = Path(f'temp/Asset_Tags/')
    if not save_path.exists():
        save_path.mkdir(parents=True, )
    nrow = 8
    vutils.save_image(imgs_, save_path/f'{tag_}_{batch_size}.jpg', nrow=nrow)

habitat  :  1
13081
24
watermark  :  13081
7357
24
archive  :  7357
2750
24
field_notes_sketch; watermark  :  2750
18
18
back_of_camera  :  18
111
24
multiple_species  :  111
1089
24
field_notes_sketch  :  1089
238
24
field_notes_sketch; non_bird  :  238
62
24
non_bird  :  62
1005
24
habitat; watermark  :  1005
113
24
multiple_species; watermark  :  113
34
24
field_notes_sketch; habitat  :  34
72
24
habitat; non_bird  :  72
36
24
habitat; multiple_species  :  36
21
21
back_of_camera; habitat  :  21
2
2
no_playback  :  2
10
10
back_of_camera; watermark  :  10
7
7
back_of_camera; habitat; multiple_species; watermark  :  7
1
1
non_bird; watermark  :  1
18
18
archive; watermark  :  18
3
3
multiple_species; non_bird  :  3
7
7
field_notes_sketch; multiple_species; non_bird  :  7
1
1
habitat; multiple_species; watermark  :  1
1
1
figure  :  1
1
1
playback  :  1
3
3
field_notes_sketch; habitat; watermark  :  3
2
2
back_of_camera; field_notes_sketch; habitat; multiple_species; non_bird; waterma

In [62]:


# print(f'{i:,d}, Data size : {len(files_path_ )},  {tag_:30s}', end='\r')